# Differential Privacy in Syft

**NOTE**: 

The DP system has changed since the creation of this notebook but the underlying theory is the same. Please switch to the [padawan_trial_dp](https://github.com/OpenMined/pysyft/tree/padawan_trial_dp) and install syft in editable mode.

In [ ]:
PATH_TO_MY_SYFT_REPO = "~/PySyft/"

In [ ]:
# Switch to padawan trial dp branch
!cd $PATH_TO_MY_SYFT_REPO && git fetch origin padawan_trial_dp && git checkout padawan_trial_dp

In [ ]:
# Install syft w.r.t the padawan trail dp branch.
!pip install -e $PATH_TO_MY_SYFT_REPO/packages/syft

In [ ]:
import syft as sy
assert sy.__version__ == '0.7.0-beta.57'

**Credit**: DP Team Lead - <a href="https://github.com/IshanMi">Ishan Mishra</a>

> "You can't stop change any more than you can stop the sun from setting." ~ Shmi Skywalker

In this session we cover:
- Motivations
- Terminology
- Building an Intuition
- With or without you
- With added noise
- How Much Noise to Add in Words
- Privacy-Accuracy Trade-off
- How Much Noise to Add in Math
- Introducing ε, the Privacy Budget
- The Effects of ε
- Standard Deviation
- Why does DP work?
- Our DP Implementation
- DP in Action
- Filtering
- Accounting
- Under the hood
- DP Tensors in Syft
- Too much noise
- Not enough noise
- Data Subjects
- PhiTensor
- Tensor Chain and .child
- Upper and Lower Bounds
- LazyRepeatArray
- Data Subjects
- GammaTensor
- DataSubjectArray
- Operations
- Add Child Data
- Add Bounds
- Add Data Subjects
- Provenance
- Anatomy of an Op
- Publishing in Syft
- Private Data
- Upload Dataset
- Create Data Scientist Account
- Private Mean
- Publish
- Compare
- Epsilon Spend
- RDP Parameters
- Calculating Epsilon Fast
- RDP Constant
- RDP Cache
- Differntial Privacy Ledger
- Data Subject Safe Guards
- Trial of the Spirit
- Brain Food
- Homework
- Bonus Materials
- DP Karaoke 🎤

## Motivations 

In a previous session, we discussed how difficult it is to protect people's privacy when working with or releasing data. We learned about the Netflix prize, where unwitting `data subjects` were de-anonymized with shocking accuracy.

We talked about the `Copy Problem` and the concerns with `Data Linkage` and  discussed potential solutions to these problems in the form of Privacy Enhancing Technologies (PETs).

Differential Privacy is one such `PET` which helps to protect `Data Subjects` by controlling the `Output Privacy` of computations.

There are many different implementations of `DP` (Differential Privacy) and the way we do it is a combination of different properties we desire to solve the previosuly discussed problems.

Some ways to describe different features include: `Adversarial Differential Privacy`, `Individual Differential Privacy` and `Automatic Differential Privacy`.

### Terminology

Let's start by outlining some of the parties involved in achieving `Adversarial Differential Privacy`, where by there are at least `three` distinct types (or roles) of individuals in the interaction.

<img src="data:image/webp;base64,
...
IDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CgA=
" />

In the above information flow we can see the following:
- Data is collected from Individuals which are referred to as `Data Subjects`
- The data is usually cleaned and preprocessed into `datasets` by `Data Owners`
- The `Data Owners` then make these `datasets` available to `Data scientists`, who apply algorithms and use the data to answer questions, draw useful conclusions and build products

## Building an Intuition

`Differential Privacy` is a mathematical guarantee that if you run an algorithm against a `dataset`, the output is indistinguishable from:
- running the same algorithm, against the same dataset again, a second time
- **except**, the second time, removing data belonging to a specific `Data Subject`

```python
# result of operation on all data is indistinguishable from
# result of operation on all data minus a Data Subject's data points
dp(op(data[:])) ~= dp(op(data[:i] + data[i+1:]))
```

The result of such a guarantee, is that it should not be possible to guess if a specific `Data Subject` is represented in a dataset. Therefore by extension the privacy of any given individual is protected because you cannot learn anything about someone, if you cannot know with any certainty if they are present or absent from the dataset used in your algorithm.

### With or without you

First let's see what the real difference would be between a query with or without your data.

<img src="data:image/webp;base64,
...
" />

### With added noise

Now let's see how we can hide that difference by just adding noise so it looks similar.

<img src="data:image/webp;base64,
...
">

This means the `Data Scientist` still gets a reasonable and accurate answer of approximately `~25.7`, but can't be sure if you were in the dataset; and if you were, what impact your `age` had on the `average`.

So in a nutshell, `DP` makes it so that the Data Scientist doesn't work with just datasets, they work with, datasets + plus noise. Infact, just enough noise to protect the privacy of the data subjects, while still giving useful results.

<img src="data:image/webp;base64,
...
REY+CjwveDp4bXBtZXRhPgoA"
/>

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### How Much Noise to Add in Words

Thus far, we've seen how `DP` (Differential Privacy) leads to the addition of a small amount of noise to protect the privacy of someone's data.

A reasonable next question to ask is: `How much noise should we add?`

Let's try with an example:

<img src="data:image/webp;base64,
...
" />

It's important to understand that the way a `Data Scientist` can learn private information about individuals in a dataset is through comparisons of results over time. In the above example if the `Data Scientist` got the `mean` of the dataset and it was `25.705` and then were able to query the system without your data (many data APIs allow for different ways to group or select data based on fields) seeing a result of `25.70` this small difference might tip them off enough to guess your age with reasonable accuracy.

Let's try again.

<img src="data:image/webp;base64,
...
" />

This time, while your age is now very hard to guess, we have added so much noise that the results have lost some of their accuracy and utility to the `Data Scientist`.

### Privacy-Accuracy Trade-off

What this should hopefully convey to you is that there is a `tradeoff` between the privacy gained by adding noise and the corresponding utility lost in the results we get from our algorithms.

- If we add too much noise, the results are no longer accurate, rendering the algorithm useless
- If we don't add enough noise, the results are accurate, but we aren't able to protect anyone's privacy

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### How Much Noise to Add in Math

DP is a mathematical guarantee, and so there is a mathematical answer to this question. But before we can answer the question of `how much noise to add`, we'll need to understand a key insight: `datasets are distributions.`

Let's explore this idea with a simple example.

Imagine you had a dataset consisting of the numbers:  
`[1, 2, 3, 4, 5]`

An easy way you could convert this from a dataset into a distribution, is if you iterated through every datapoint in the dataset, and asked, `what is the probability of number n being in this dataset?`.

The results could be shown as a histogram that looks like this:

<img src="data:image/webp;base64,
...
" />

Voila! Using this simple scheme, we've converted our dataset into a `probability distribution`.

What was the point of that?

Why did we need to convert our dataset into a distribution?  
Well, it turns out, there are lots of useful ways to compare two distributions!

For example, here are sixteen popular functions which can be used to compare two distributions:

<img src="data:image/webp;base64,
...
IDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CgA=
" />

This means we now have a well defined way to `compare any two datasets`, by simply converting them to distributions first.

So, why compare datasets?  
What does comparison have to do with figuring out `how much noise we want to add`?

Well, let's reconsider the definition of `Differential Privacy`:  
The result of an algorithm is ***similar*** when a single person's data is removed from a dataset.

Therefore, if we know how similar the results are, we immediately know how much noise is enough to make them indistinguishable. In addition, calculating this similarity is much easier if we treat our datasets as if they were distributions.

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### Introducing $\epsilon$, the Privacy Budget

This "similarity" or difference is captured in a parameter called $\epsilon$ (`epsilon`), which we often refer to as the `privacy budget`.

$\epsilon$ has different formulas (depending on which method we used to compare our datasets or distributions), but it is always a measure of how much your data (belonging to a data subject) affects the algorithms result.

<img src="data:image/webp;base64,
...
" />

$\epsilon$, or the `Privacy Budget`, is probably the most important idea in `Differential Privacy`, so it's worth taking some time to emphasize exactly what is is.

The privacy budget, $\epsilon$, is a measure of:
* How much your data stands out
    * Thus, it's also a measure of *privacy risk*; how likely your data is going to be identified
* How much your data affects the outcome of the query or algorithm
* How much noise is needed to hide your data's influence

**Note:** 
There is a more rigorous mathematical definition of epsilon (see resources below), but thinking about it in this way helps to build an intuition.

### The Effects of $\epsilon$

**The higher the $\epsilon$, the more your data affects the outcome of an algorithm.**

<img src="data:image/webp;base64,
...
OlJERj4KPC94OnhtcG1ldGE+CgA=
" />

For example, imagine if we're trying to calculate the net worth of all the people living inside a single city, and we did this for every city in the United States.

Warren Buffet would have a gigantic value of $\epsilon$, compared with nearly any every other person.

**The higher the $\epsilon$, the more noise needs to be added to protect your privacy.**

This should make intuitive sense:
* As we discussed above, the higher the epsilon, the more your data affects the outcome of an algorithm
* The more your data affects the outcome of an algorithm, the more noise you need to add to mask the effects of your data

**In theory**, noise is a random value that depends on $\epsilon$  
**In practice**, we obtain it by sampling from a distribution where $\epsilon$ is a parameter  

### Standard Deviation

<img src="data:image/webp;base64,
...
bXBtZXRhPgo=
" />

A standard deviation (or $\sigma$) is a measure of how dispersed the data is in relation to the mean. A low standard deviation (like the image on the left) means data is clustered around the mean, and high standard deviation (like the image on the right) indicates data is more spread out.

To hide any real data point with noise it needs to move along the `x-axis` to blend in with distribution and be closer to the `mean`.

Therefore, the larger the $\sigma$ (Standard Deviation) of the distribution the more noise is likely required to be added in either direction to hide it amongst the other data points.

We said $\epsilon$ is a measure of the risk to your data's privacy, so the bigger the risk to your privacy ($\epsilon$), the more noise ($\sigma$) needs to be added to protect your privacy.

Often, it's a normal (Gaussian) distribution where $\epsilon$ affects the standard deviation ($\sigma$).  
So the bigger the $\epsilon$, the bigger the $\sigma$, and thus, the wider the range of values the noise will likely be sampled from.

We know this because of the empirical (`68-95-99.7`) rule in statistics, which says that when you sample a normal distribution, approximately `68%` of the values will be within one standard deviation of the mean, `95%` will be within 2 standard deviations of the mean, and `99.7%` will be within 3 deviations of the mean, and so forth.

This rule is also how the company `Six Sigma` got its name! They aim to get manufacturing processes error rates to a six sigma rate, which essentially means no more than `3.4` defects in every `one million` fabrications.

<img src="data:image/webp;base64,
...
dGE+Cg==
">

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### Why does DP work?

I'd like to leave you with a intuitive, gut feeling as to why `Differential Privacy` (this method of adding `noise`) actually works to `protect` privacy.

The intuition is that when it comes to protecting privacy, your data "hides" in the background of other people's data. The more your data fits in with other people's data, the easier it is to blend in and hide in plain sight.

Put another way, it's much easier to spot the horse in the image on the left, than the image on the right.

<img src="data:image/webp;base64,
...
" />

<br>

Hint: Look for the horse's mouth directly underneath one of the wall lights.

Again, the best way to hide a data point, is to surround it with other, very similar data points! They'll be hard to tell apart, and it'll be like trying to look for a piece of **hay** in a very large, fluffy haystack:

<img src="data:image/webp;base64,
...
PC94OnhtcG1ldGE+CgA=" />

By adding noise, we're doing something very similar, we're making it hard to discern one data point from another; the same way that dumping all your hay in a haystack makes it hard to find a specific piece of hay.

### Takeaways

Differential Privacy:
- Allows you to learn `high level statistics` and trends about things like:
    - How big is the haystack?
    - Am I running out of hay?
    - Is my hay on fire?
- But, `not` specific statistics about `individual data` like:
    - is the piece of hay in the center getting moldy?

This means you can:
- Answer questions such as `What causes cancer?` but not `Does Walter White have cancer?`
- Differential Privacy works by adding noise to the output of an algorithm to protect privacy
- $\epsilon$, the privacy budget, is an indicator of how likely it is that someone's data stands out and can be identified:
    - The higher the $\epsilon$, the more likely that person's data stands out and will be identified
    - $\epsilon$ can be calculated in many ways, but it's always an indicator of the risk of being identified and having your privacy violated

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## Our DP Implementation

The goal of this section is to help you understand the major components of the `Differential Privacy` system in `syft`, as well as how Syft's DP system is differs from other DP systems elsewhere.

The Differential Privacy in `Syft` is different in 3 crucial ways:

### 1. It's Adversarial DP

- Data which is `private`, can be safely studied by `Data Scientists` who cannot copy the data
- The same people tasked with getting maximum utility (`Data Scientists`) from the data, are not in charge of evaluating the `Data Subjects` privacy risk (`Data Owners`)
- There is a hard limit on how much any `Data Scientist` who interacts with a dataset can learn about it
- This limit is how much privacy budget ($\epsilon$) the `Data Owner` gives to the `Data Scientists`

For example, say the Data Owner gives a Data Scientist 5 $\epsilon$ of privacy budget. Every time they want to get a result, it might cost them a few $\epsilon$, and the system would deduct this from their available balance. Once that `balance` runs out no more new `results` can be downloaded.

### 2. It's Individual DP

- We track the privacy risk for all `Data Subjects` in a dataset
- This is useful because by knowing exactly how much each person's privacy is at risk, we can choose to remove only their data from a computation, if it gets too close to violating their privacy
- This gives `Data Scientists` more "mileage", since being able to add `less noise` to any remaining data points is far better than adding an extreme level of noise to all data points just to hide a high risk person
- The more `risk` the Data Scientist's algorithm poses to the Data Subject's privacy, the more of the Data Scientist's $\epsilon$ gets deducted
- Accuracy is fine-tunable by the `Data Scientist` so that a less invasive query might only consume 2 $\epsilon$, whereas a query with higher privacy risk might consume 10 $\epsilon$

This is in contrast to regular Differential Privacy, where an entire dataset is characterized by a single $\epsilon$.

This is because regular DP only considers the privacy risk of the individual whose data is most at risk. It's much easier to safe guard your `Data Subjects` from a `data leak` if you always assume the `worst case`; however, in the end this means less total safe `utility`.

### 3. It's Automatic DP

- By keeping a log of all `Privacy Budget` on a ledger and allowing `Data Owners` to define safe guards for their `Data Subjects`; we can know if the noise added to a result for any given `Data Subject` fits within these safe guards
- If the query is within privacy budget ($\epsilon$), a `Data Scientist` can automatically `get` the result without waiting for approval

This is very different from most systems where the `Data Scientist` usually needs to submit an application for their results and the probability of `Data Subject` risk has to be determined by another person by hand.

<br>

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## DP in Action

Let's say we have a `Data Scientist` who is given $\epsilon$ = 10 of privacy budget to work with.  
The dataset they're using has 3 people's data in it: `Rob`, `Bob`, and `Job`.

<img src="data:image/webp;base64,
...
" />

Let's say Rob and Bob's private data is both `4`, and Job's data is `6`.

Although this dataset is small, we can easily see that Job's data `sticks out` more than Rob's and Bob's. Thus, we expect his privacy to be more at risk, and any epsilon for Job to be higher.

Let's say the Data Scientist wants to do a `sum()` operation.

<img src="data:image/webp;base64,
...
L3JkZjpSREY+CjwveDp4bXBtZXRhPgoA
" />

Now that we know which algorithm we're evaluating (a simple `sum`), we can calculate the `epsilon` values for every Data Subject in our dataset.

You'll learn how we calculate this very soon, but for now let's say the `epsilon` values for `Rob`, `Bob` and `Job` are `6`, `6`, and `12` respectively.

We can see since `Rob` and `Bob` have the same private value, their data stands out just as much when doing a sum. `Job`, on the other hand, has a larger data point which will affect the result of the sum more. This means his data is more at risk, and thus his `epsilon` should be higher.

The fact that `syft` is able to assess and treat privacy risk and compute and store $\epsilon$ values for each person in the dataset is an example of the `Individual` aspect `Individual Differential Privacy`.

<img src="data:image/webp;base64,
...
Pgo8L3g6eG1wbWV0YT4KAA==
">

Now that we know what the individual `epsilon` is for each `Data Subject` during this `query`, we should now be asking ourselves, does the `Data Scientist` have enough `privacy budget` to get the results of this query?

Remember, we only gave them 10 $\epsilon$ to begin with.

<img src="data:image/webp;base64,
...
cG1ldGE+CgA=
" />

A quick comparison of each value tells us that the `Data Scientist` has enough privacy budget to utilize `Rob` and `Bob`'s data:

```
10 > 6  == True
10 > 6  == True
```

But **not enough** to use `Job`'s data:
```
10 > 12 == False
```

### Filtering

What can we do? Well, let's think about it. 

If we let the `Data Scientist` use `Job`'s data as well, this would be a violation of `Job`'s privacy.

The system is telling us, to guarantee that `Job` is hidden we need at least `12` `privacy budget` to be spent when calculating the noise added, otherwise a `Data Scientist` might be able to figure out `Job` is in this dataset and even learn something about his `private` value.

So, what we could do instead is remove `Job`'s data from the computation entirely. `Job` wins, because his privacy is protected and the `Data Scientist` wins because they will actually get a result to work with automatically.

The limit of 10 $\epsilon$ that was given to the `Data Scientist` represents a strict cap on how much they can learn about any specific `Data Subject` and it's critical that these limits are enforced. This is an example of the `Adversarial` aspect of `Adversarial Differential Privacy`.

But `Adversarial DP` alone would not be able to proceed with the `query`. By combining `Individual` DP and `Adversarial` DP, `Job` the `Data Subject` is protected but the `Data Scientist` still gets some utility.

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

<img src="data:image/webp;base64,
...
eG1wbWV0YT4KAA==
" />

### Accounting

After filtering we proceed with the sum calculation.

The sum of `Rob` and `Bob`'s data would be `4 + 4 = 8`.
The result then needs noise to be added based on the highest (`max`) epsilon value calculated for the remaining data points. 

In this case, let's say we calcuated noise of `0.8`, the final result would be `8.8`.

To allow the `Data Scientist` to access this published value, we need to account for the spend by deducting the cost from their available budget.

How much did this `query` cost in `epsilon budget`?
The `epsilon budget` is the highest (`max`) epsilon value of all epsilon values calculated for the remaining Data Subjects's data points.

In other words, the `privacy risk` of a `query` is equal to the `highest` risk posed to any individual in the dataset. In this case, the max `epsilon spend` was `6`.

```
current_budget - epsilon_spend == remaining_budget
            10 - 6             == 4
```

This `accounting` of all the `epsilon` values allowing for the automatic release of data + noise to the `Data Scientist` while safe guarding the `Data Subjects` privacy, is the `Automatic` in `Automatic Differential Privacy`.

<img src="data:image/webp;base64,
...
" />

<br>

<img src="data:image/webp;base64,
...
" />

### Syft's DP system is
- **Adversarial**:
    - A `Data Scientist` can learn no more than what their privacy budget allows
- **Individual**:
    - Privacy budgeting $\epsilon$ is tracked at the level of individual Data Subjects.
    - While the privacy budget for a `query` is the maximum epsilon of any individual Data Subject whose data was involved in the query, we can also `filter` Data Subject's automatically to lower the cost
- **Automatic**:
    - If you have enough privacy budget, you can get the results of any query you want without waiting

### Importance for Data Owners

Image you were giving a data scientist access to a classified dataset.

How do these features help you as a `Data Owner`?

- **Adversarial**:
  - `Data Scientist` has a hard limit on how much they can learn
  - They are incentivized to be frugal with their query costs due to the hard limits of their total budget
  - Lower cost queries mean lower risk to `Data Subject` privacy
- **Individual**:  
  - You have a guarantee that each of your `Data Subject`'s data will remain `private`
- **Automatic**:
  - You don't have to get on the phone, approve every line of code they write, or fill out paperwork

### Importance for Data Scientists

Image you were the the data scientist getting access to this classified dataset.  

How do these features help you as a `Data Scientist`

- **Adversarial**:
  - The safe guards in place increase the confidence of `Data Owner`s and mean it's more likely you will be trusted with acccess to important datasets  
- **Individual**:
  - You can do your work without worrying about accidentally violating someone's privacy because the system won't let you
- **Automatic**:
  - You don't have to get on the phone or fill out any paperwork, just to get permission to do your work

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## Under the hood

Differential Privacy in `syft` is made up of three main components:

- DP Tensors
- DP Ledger
- Publishing

The good news is that you've already met them above.

<img src="data:image/webp;base64,
...
ZXRhPgoA
" />

In our `Rob`, `Bob` and `Job` example we private data that we wanted to do `Data Science` on.
This is handled by `syft`'s **DP Tensors** and acted on with our `Pointer` system so that `Data Scientists` can't copy the data.

We talked about how `syft` tracks and differentiates individuals by recording the history of their cumulative `privacy risk` in something we call a **DP Ledger** which lives inside a `Domain` along side the `private` data.

Finally, the process of calculating `epsilon`, filtering out data which is overbudget and returning a result + noise to a `Data Scientist` automatically is something we call **Publishing.**


<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRjAjAABXRUJQVlA4WAoAAAA0AAAAYwAAYwAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBIfQQAAAGgRW2bIUn6IyIjI9W2Z23btnds27Zt2/aV
bdu2PTtqVVbGRXcp89/7iJgACCd/Y8btp9vKWAAAaqnThatfJpRSIXSdazGxlMZG
a1zXhUYJJRCkVWbb09sz3uDgTvb7WSmlzO0XBwD/3JNSns0RKqcAAIoZl5AYZzIA
AKqqQnBOiovrlyullGf/YO5I2+AUkfdKElBHO1JKWR5Afe2fFoNnL129fuPG9auX
zh7c/J/XdADKFN1gAEBK3pNFnQ1p7vjML4vfxUCfJIt2aLbqsc/2OzJIx2/77q9s
+jwBANXgwHbJ4v2fuqOCDNwhlpe8WSSCvqODfnpeAx7XXgau4I4f/YHsD6JLnY+Q
lDJ/59Dfoz6wA/l/cMezxwPJbVtv+yMnpX1n+w4Z+Piz7uCdCgN5sbATdwe8vMdL
e14Gl6q9vdRbdQmzrD3e2WtZzA1EMVmJeZ5xZmVTk5OIUSHgm3U+z0i55TcuNBop
TZBqlxzp5YvVuG6RyHAmWhdIj+fVUrhKIkA1ZnX+T3re1yWBWzR8QojBDyWCuf3j
dY2ESxF8WKFEMa8LNdQwMVNr+0QiaddTTRYWZpFK1yWa+TVVi4WBaOy9KxLRCz8r
GgmN87TrEtVtWZyHxLS4mQ4ucqwQLBRTq31fIltQn5khCJ51SqJ7/T0ugiKmMkXi
68yKNkkwgv7lQ0jeKslEEExL3y1RXmUKFkgRTXNxckqqSiA9e4NE+nSMEUDhFZ9g
JRuoSnFRdJVEe2diVDFMf8OP193SOitikAUSb2eMaQAA0Z/NRUweflEnAFxpIjH3
/6lwAG4sRU1OZhyIeOsYblc0lVBR+j5uzueCUr2zg5vsrxKaPEcif0BQLXsvdo/S
hXjxPnZ5XwvjK4l9YS3dqoOe3csw+6HnTNOsWejJ5XrCavx2xqbswe9Ycvpx/M6n
pZ/F70pa6v9BSuIJ/M7HR+3D74RprcVvlxE1F79VhjUIPWemrjdCzx4g9J/QK2wg
6FsPscv7jZLnDmP35FlCUxdjd0ajxOrtIDdGJcCrPETuNwWAf3Qat4c6ByApq3Fb
qBMAMDqh5q9kAACwd1E7+x4rArGbEXOmZ0Cx6q82Xg9r8eIgdideh5+HgEp9B63e
SiD2xl6sbuawQBDVLR8np7mAIMmrF3Dak0GCAdHCxuhpbR2CT1yLkLPkeQiRfXYF
n4e/K6GA0cmHzkANQibPr3BwcdbEkNCAfXcdl8NfMAgn/ycPkzu1BISXt83Hwx6s
Q5hJ2ug8LAqmxJNwAX1hDhK+SZkUwk9ilqLgX5hJIJIkdV6e9wqXPE8gsvS5cYVe
ezw2g0KkSXK7Am/91zWRgAtZhXteyq3HwZ3m6Fzv5A3Twa1KmeOOR06UUcDFyUNt
L9iDUsHlbx2x3WYfegPcr9Y95HOTfbihBp5MKbfJ57jEt7VCMng3q8lx24mU4zvQ
MBU8zr8fsvF8bvjyz28e8r0BCNKMb+uPWnvVDs1/df3Iel+nE8CSmok5H1frt2Dr
qbt5tj/37tltC/pX+zgnyaTgTgBWUDgg3A4AANA7AJ0BKmQAZAA+MRSIQqIhIRXs
BzggAwS2BtgCuCjVfvot96/Ib2SrB/Xvw3+U3Nbl37NP239w/ID58/431LeYH+ov
649ZDzAfs7+w/u/f7j9gPcx6An8p/rn/47D/0HP289Mj9t/hF/b/9vvaA/9/WAcA
12q/4H8hvQfxqebfZ3+jekr40OofEt9sPxP9r/bT2V78+AF+P/zf/Cfl3+SnIOgA
/M/5x/sfuO9Mr/F9CfsJ/rfcA/k39I/3Hqt/vvEX9H9gT+f/3P/nf4X2Cf/D/Qee
76U/aX4Bf5n/YP+j/gf3e+MD2KfuT7G/7AJ43H32bIPfV1ZQSdoGU92JRmfrtz+q
0puw85yx4qrz5TsSH5x912iBlHEY+xfneluU0vwSjMnGHRFgXwXXYALK8k9UMC5v
Wgr3TXDuJ7Stmo66PVjwbXmazL4L2TWOJvmrUqbu/3lTMEQEQLcHIojoMNcsnqDr
NMUm0fB9BaGyW1UAEoFmetzNftAcjTSswDVBe+1AAYqXOvgdQfzURAIWh1deoR6s
D7Xl/6FHNcR17zk1W54Zf7baf4NzJhd3qE9kSR5xWLTV8YcEbYArguPEViwOZ6Eh
ImDMY3FiSMRgvOtzxbug5cSYEsc2xucIQonf2IZfQBg9wgAA/v/oN2X8R59hrAX6
uBisLe/1gGv6JofL4j24b6pvEW79wOzdnebLYvFoZuatF8/dsZrJp9SfhqixZR2S
1POFQA99w8/eTy6bL0ppRLrRJkEdoklX1ehWjO2Sf69mDb3T/zMJCT5skZaLyrRf
b2fqX4Kv/e2EjerKV+xCdNLVtdU+ktKmUbhaTZrRLrxj+ukW3jkRO7VnQI0ciodk
8ZI9lUBUBoQdHob9TwRJn5DXFPcJxUi/iD1pDebSNf3Pz2/EhmWblMHLCP+Zn0LW
Q+F9oupaUV2zCU8J5GU9KRmQg9eEvdU1HXcL+IHck07Mf5YfBx02EXY5ZD/H9X3Z
ZcI358QI7zmma7wznB7SKuc9RCbi6SY/FLR69yxvKs69FTSRdoa40llqDaEuYs7y
ge0rLb4TFAygGVcDjXJqpaqYfXqSCbkd1RrZii+2dRv1UnaPAfqhEKpzAJaUIFL7
A6CgSItms+Q9iPkPjBRHTnduBcAR5OJI7Vd6eYx3GpX3vXOlSK4KcwFvLJWeG+ta
2nbdD7FnKJOu/9q9aU0UbKo0nOST7/fwT4lPxnKyFEr+XiFoK2XH/FtuYHmh1qtL
+b6gqfZbeoE8wMmnClofgnqxJc812yKyRaQF236q+a86h1TlAwifFff7NauX0fo/
07cNW/g9oaJAHKyHxKa21pLsw/HZqKv55N7DeqhFnNxWO5j8SB6PanfmZMeK/CKE
DyXqB9xbvUlNfJGC8w/lN0gP0aoqGC4Z4ty2vm765aYaiZuMCQVRJt/KnVPxDL79
w0enghM/6dsaCYd6QS4WiAMbYyVZtVTWA7MRoSU0iapfvwILJOAp/CN/N97y/7fA
mCiYW1o9tFnDZw15OsQw3YEkHLFD6F8Tp2d4+tD2iIfhBfLWzs/psgs1C2KVqEVu
M/+j4VBAyi6MMJzFjBSkuehPMz7BbfwDMY03xT4ae4ejNRCGB+KSaiN3Rk750NiW
V1n7s6ZxfCwlmHfAFTf86Y9IgE2JEVmg4Iox6c9+eJLwkMCvcrI1nqnTItHO9f4P
rEsWG/hVeRx4eSanSx3HSqPHH1NZlPuJDvBe+ilXBKggPrK9yU0aAS/rwO+GKmTJ
9mIeZlUIJ4Af7bqLdfG1MmBZvI3AD3TorjAzmZ8aeWKSTALuWCfl1TItVsulOFAa
xT1Kc70TZAqc1Ip1xlXUzQtCbd+4BQssGbz0lQnfpj4rK2tE/yjyqdMfAeWU12ik
flaoM9St/cshxMy9W1bc9Ygi8g+/pMIPBgS1I4FbMAmKB7qvBTgX8Frt3fWfe7iN
L/7bfykEXCD3qIhNoplnPGeqid4idf8tRkI5UFTdDrJgjaHsSkhjhSTjzjhYcNZ/
pDGvV3iSTFe+5Kcb7C/fFkR3YpRBtQT7IirMlLglMinwGTBEP54woseL2FEQfhjV
cbVXTMRP08A1ULLqOewo5qooICvj0///w/jl5/z3TppNaMwSXmKsX/CpWz4d/SEO
wIOBU7+1LEzlRmIRzuHkGtBy6rh8n91Kov8ilcEdCGVgcaZdPkLZSFxR10RB60Cg
x4ntcQ3I4ojyxpKe8kCY8m/fcBcoPvGSCheGmf5fvUmC6LxC/61Gna7iFJwuwMRl
bTuSYb6sVm5V806xp0GUKXy29uEyrdTjXpPiA4FtleHXVZb7V1jb16Gt3EBuGgb6
RzX8f8d6LThACaJqv7nAeyJ1WRnAYJ2CBCuR0MlGuQWd3cg0MDL+f+H7griBqGe2
HapROc1TpIIecVsaidl+/m/w/kW3eLHIMjJ5cbF65iSIFqJkQJrH5fn0QEa9IVpW
ZVznDxaEoL6NvFYriWTTum4la/R9Facts0BjeDotKCI/nWu6P17bCmNPnWdiQYIL
gp7jPUlakpv5QaT+w3Y+W7hfC2t/jLnCO87QDhubrasFsZLC+DiLgFS2n/+6NLYh
tB/dYGYAZVOgQWkWO+L4Cny47XdPEF5z9q+OmMGwXoxNQxePR1+j4FJQvWeA5OJU
JEZmCJUtUfIUwfHqEBT8y4b32LchlDnGbHdXaZ7BCR/QeRpMY5Uf5+Xgbfva3oWS
CoC1hvvk8fnnn/59OC3oWMhEh0B2J2SVXsc/IxubBBhT2qO4UOEGZAVqhzdHLm9F
7Q8aNM1e8uFTsFZmr2WCfiZb0f4951fFqxcAKAoS16HK1a/8WfHBP3mHGqtcRPmh
K3/xir4rd9CRqdBuu0oiciD/mdl412Jk2CsWm2aFuoDxsx8kvwYAaoetI/BlxZc0
SVsoIB0sDnuUuW6IrkKZnXgC6rTjsvu9DjJ9LTES+s5AmjK5d4mestcoWOUDYhxv
ZGvcgkCfDTTu8pVFiRyDAQjWp++zNV8EkzMPi6aJe9OHVJUrgUSgNgwE5nlxCaGv
lBobUYrHOhSptiLQ/vDKm+AlSm3G4j8GQBF+/+pTFgwIJL6hSWu/nf/dAIWSAOR6
C5RpFdOsltnNcrW1qWM9gJa9aLcVdI6vjzTb+RNLMQ0SAzssFGPeq+R7Fzcub0vt
bd7m2Fl9HHnIDotd4e/4ZP9DrpHJ+o6e80R/NnMw3nrBE+5MMHl3HPUyK4bcZJev
/LHCc5mh+Oo5HL2COv3zXykbg2q9FP+Y1dBjH0x9YV5o/7jV3vnl9Xz4jjbTCIIS
4Ss2anEFQckyc+LjQM2liXWTRSDytwHOA5cS6rWz3dkdpy+u2tst4nAuPlhXQpPs
0UujsixQ/absVYL9440fRxO2oVBvIAcLHzjsqnBx9DUqmPtfapaosaOmO1UVnt4P
XGiRnd4/mqMnOJ9kZM/4CneOG4i2wgbJAkURGDiyzbPPSa+E9YDFDuDKI0Wp/d1C
x/clfOHNNA/phW1bQKhDtmLDXn7wpdCS92n1AJKA7sCWluwPTXgJB7i1Q1kvqmmM
Cu0KGVLoTZwgRQYW2Ne8nFFFJtbH/X4cK/kjjQxqsdnWLIzko4L1iMr06E9DyuYf
d9W/6XAPBnsv62NhGUuQ6WfxCctD65xfXERD8hxJvHKaAa7/lM6W4/y+DD7E/n+a
XPGJgiOKP6O1fHxggrey+DqDkPUvWrgZ3LFfV1Zf7v/7FnMvnNy9S2eklrOe/rJ+
0/o9VyeqMT7p/NRm7yMdS0Q8oczaPRAfPjzkX4Op9uAu/2n8XuLhVJW/MaxgWkzp
T7KJBNOYgMGxU5ikiITE7KQqdmOcoUxAmAURV2r4QHcRVO9DxefWl+vv59Gfyjwk
1vYvNnbSwoulILAnp7pykZlQOgOnu3Lo5ySpjAcPFDGbZJm6q32peXWabycDbgCH
Gf4Owp5EhBWTJJ0z6hiW8fIbUlML3IAgEhzERx/jZiF7jF+F2fU7ZfZrBlYz1HLD
mNOOWryrenAeoLjvpyk380G9MifioUwG8wu+oLL4lwrBp3bkUjqDJweBJx6XElCh
YXbU8RNduQ7M6+3BRD2WgvPVeFs+vL7JhGihKZg9t1987iF+5I7GPbeTUqIUqEIP
RMHdaZhJE42leiRhVs+xm15o4MX1MnzXy2UTSdbjt+S18sUF8PiXpG/aFJrTsnyP
8B4IJjwujaCw3NhUxvUPlUP1ZvOOQHpXrSl/pHAd8ZLsWwswJnc5mJTmI4JzgZ9U
/4E772jFeA56o1iWE3osT//2yLg1eK10GAxRR5a0AoA0jCFtumufv+b5ZuSR6yaI
YSUbbcMrtmZXYUOBpgBGQ47U9wXLeAxT8QpBkoEE7WFBdxeFNRNucYuFpehKv1QR
Ht0o8RyTjiNCLL58GIZUV/du4cKXtoTDMvxXfHZPycDaPTA4jq0779ELL7gAVOig
IKnHEdfYazkwOwXE0u1N6076k7YfERWeASgiQ1mtfrt7KGzE9nEmftVcgRsdEGZY
PgwV1yS6VU1oa5LDtMpYoYwl1uX0kTj7IBM7U83zk+Sb/3o3Di/D3Xws3Of8SlrJ
sn/5xh5ARZPXu0TNbZEkCVp9C0Jxgr70Iy9KH3hRo0DiTa98rXVIpIYzfREvhwVC
VBMDW41FPScjqw7GHO31G5vQCss56385Dm6Zdyz7IRZlVbEge//ggYe/mYCXxjHw
6kK8hSl8W/HSgWMGxtk5qLdXl/6mLPanzbql4LYwT6Jb6fsvNMWPDXWZdpCVkdJY
UjhM5PFEDZtE1ApUYX+0waQlrpm2A7JQoiDjSs1MrvxcKJ35eM+Q+aWnISlL/KV6
E2pH/wDDwWHq6cnA7rSEE1eNVMwoQHdN6etMlbYWYha+VVqFEGznw9DPE5eBYgAA
PdvtrrXX0CbwtqeNjHoXgIuy7nFG2G9NlLxHIubHjxxzzeZge0HF2uF9Nw6+RBTb
uDQGAIw4Qa3jXxVdnhNBmAECrbUFfo7xHN2lWxhWjzBbhCsmfKsti3G6Sm9/7ik/
/4UqJK5VbgpHViAObOe56CxjuKSnlfZdZ3dz1mCaLDE3ottwAAAAAFhNUCBYAwAA
PHg6eG1wbWV0YSB4bWxuczp4PSJhZG9iZTpuczptZXRhLyIgeDp4bXB0az0iWE1Q
IENvcmUgNi4wLjAiPgogICA8cmRmOlJERiB4bWxuczpyZGY9Imh0dHA6Ly93d3cu
dzMub3JnLzE5OTkvMDIvMjItcmRmLXN5bnRheC1ucyMiPgogICAgICA8cmRmOkRl
c2NyaXB0aW9uIHJkZjphYm91dD0iIgogICAgICAgICAgICB4bWxuczpleGlmPSJo
dHRwOi8vbnMuYWRvYmUuY29tL2V4aWYvMS4wLyIKICAgICAgICAgICAgeG1sbnM6
dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAg
IHhtbG5zOnhtcD0iaHR0cDovL25zLmFkb2JlLmNvbS94YXAvMS4wLyI+CiAgICAg
ICAgIDxleGlmOlBpeGVsWURpbWVuc2lvbj4xMDA8L2V4aWY6UGl4ZWxZRGltZW5z
aW9uPgogICAgICAgICA8ZXhpZjpQaXhlbFhEaW1lbnNpb24+MTAwPC9leGlmOlBp
eGVsWERpbWVuc2lvbj4KICAgICAgICAgPHRpZmY6UmVzb2x1dGlvblVuaXQ+Mjwv
dGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAgICAgPHRpZmY6WFJlc29sdXRpb24+
OTYwMDAwLzEwMDAwPC90aWZmOlhSZXNvbHV0aW9uPgogICAgICAgICA8dGlmZjpP
cmllbnRhdGlvbj4xPC90aWZmOk9yaWVudGF0aW9uPgogICAgICAgICA8dGlmZjpZ
UmVzb2x1dGlvbj45NjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAg
ICAgIDx4bXA6TWV0YWRhdGFEYXRlPjIwMjItMDktMjZUMTU6MTM6MjgrMTA6MDA8
L3htcDpNZXRhZGF0YURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4
ZWxtYXRvciBQcm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6
RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">⚔ Runnable Code</h3><p>Much like training droids, these runnable cells are designed to aid in the learning process.</p>
</td>
</tr>
</table>

### DP Tensors in Syft

The first and most involved component in our DP system are our custom Tensor classes.

The biggest difference between our Tensor classes and other common `tensor` or `array` implementations such as `torch.tensor()` or `np.array()`, is that our Tensors not only store the regular numerical data, but also store sidecar *metadata.*

This is a regular `torch` tensor:

In [1]:
# ⚔️ Runnable Code
import torch
torch.tensor([1, 2, 3, 4])

tensor([1, 2, 3, 4])

This is a regular `numpy` array:

In [2]:
# ⚔️ Runnable Code
import numpy as np
np.array([1, 2, 3, 4])

array([1, 2, 3, 4])

As you can see, they both store the same data: `[1, 2, 3, 4]` and allow you to easily do math operations.

In [3]:
# ⚔️ Runnable Code
torch.Tensor([1, 2, 3, 4]) + 2

tensor([3., 4., 5., 6.])

In [4]:
# ⚔️ Runnable Code
np.array([1, 2, 3, 4]) + 2

array([3, 4, 5, 6])

The `tensors` in `syft` also provide the ability to store data:

In [5]:
# ⚔️ Runnable Code
import syft as sy
sy.Tensor([1, 2, 3, 4])

Tensor(child=[1 2 3 4])

<br>
... as well as do arithmetic on them:

In [6]:
# ⚔️ Runnable Code
sy.Tensor([1, 2, 3, 4]) + 2

Tensor(child=[3 4 5 6])

But with the addition of metadata, our `tensor` objects can help us figure out how much noise we should add to protect privacy.

Specifically, we track 3 kinds of metadata:
- a theoretical `lower bound` on the data held in the tensor
- a theoretical `upper bound` on the data held in the tensor
- the `data subjects`, or the people whose data is stored in the tensor (and thus whose privacy we're protecting)

This metadata is provided by calling `.private()` on a `sy.Tensor` like so:

In [7]:
# ⚔️ Runnable Code
sy.Tensor([1, 2, 3, 4]).private(min_val=0, max_val=5, data_subjects=["Ishan"])

Tensor(child=PhiTensor(child=[1 2 3 4], min_vals=<lazyrepeatarray data: [0] -> shape: (4,)>, max_vals=<lazyrepeatarray data: [5] -> shape: (4,)>))

By knowing the `lower` and `upper` bounds of the theoretical distribution, we can calculate the right amount of `noise` to add during `publish`.

### Too much noise

For instance, if I `sum` 10 data points in the range  of [0, 10] `(lower_bound=0, upper_bound=10)`, and I then added noise of `300`, that noise would skew the results so heavily that the signal of the true `sum` would be mostly lost.

In [8]:
# ⚔️ Runnable Code
signal = sum([1, 2, 3, 4, 5, 3, 4, 2, 1, 8])
noise = 300
result = signal + noise

print("Signal:                =", signal)
print("Noise:                 =", noise)
print("Final result after DP: =", result)

Signal:                = 33
Noise:                 = 300
Final result after DP: = 333


### Not enough noise

On the other hand, if I `sum` 10 data points in the range of [1,000,000 , 10,000,000] `(lower_bound=1_000_000, upper_bound=10_000_000)`, and I add a noise value of `50`, that's probably too little noise to meaningfully protect anyone's privacy.

In [9]:
# ⚔️ Runnable Code
signal = sum([1e6, 2e6, 3e6, 4e6, 5e6, 3e6, 4e6, 2e6, 1e6, 8e6])
noise = 50
result = signal + noise

print("Signal:                =", signal)
print("Noise:                 =", noise)
print("Final result after DP: =", result)

Signal:                = 33000000.0
Noise:                 = 50
Final result after DP: = 33000050.0


In signal processing this is often discussed in terms of the **signal-to-noise ratio** (SNR). The first example was a high `SNR` (generally considered bad) and the second example is a low `SNR` (generally considered good).

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### Data Subjects

In addition we can't do any of the previously discussed fancy `Data Subject` privacy tracking if we don't know which data points belong to which `Data Subjects`.

If we're able to keep track of which data belongs to which person, we can quantify in strict mathematical terms ($\epsilon$) how much that individual's privacy is at risk.

This is tremendously useful when it comes to things like Linkage Attacks (refer to Session 01 - the Netflix Challenge example).

Based on the number of Data Subjects in a Tensor, we can split our DP Tensors into two distinct types:
- PhiTensor: `1 unique` data subject `per data point` in the Tensor
- GammaTensor: `2 or more unique` data subjects `per data point` in the Tensor

Let's try playing around with them!

#### Underlying Data

First, let's say we have some data in `numpy` format.

<img width="500" src="data:image/webp;base64,
UklGRpotAABXRUJQVlA4WAoAAAA0AAAAiAIALAIASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBImgAAAAFnoKiRJDW5DPdiMHASLiICxMuv/a2194Wb
2LadCuiSDASgCR1IyRpoaTOICz+siP5PQFAe191b5UFxXvfvFI3XHRzPt1C23UMX
//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef
//Ef//Ef//Ef//Ef//Hfrzv+O+6h5R4abqGkOG+gOgRWUDggKh0AANDDAJ0BKokC
LQI+MRiLRCIhoRBa3OQgAwSyt3sgNID9AIkA4k5WLBKmNP/cP7H+pv7/7atz7+nf
qh/gP2G6b3Y/up+S3Yk1L5Y/D39g/pv9+/1H9s////u+338f/oP8A/u//I/w3//+
z/54/vnuAfpx/cv7p+3/9P//////OHoX8wH8x/oX+q/rf73/9z6f/67/jP69+//y
H/XD/Qf0D+7/IB/Hv6f6Pnse/2j/ZewF/F/7F/9v3n95L/y/3T9///b9mH7Sf9//
O/8r///KT/df+j+evyAegB1Q/UP+o9pH9b/J792O248S+33Jd6b8zP5R9R/t/9O/
aj8j/iX+7+HvAC/Hv5t/jPyu/MLj29C/aX1AvXH5r/wPuK+KL2n/hej31u/z/uAf
zL+b/6v1M/2X+W8h77T/qvYD/kv98/639+9dz/d/vX5Ze4P86/yn/g/0XwG/y7+r
f77+/fvf8RXsQ/dH2Jf2GE46NdX/GlUi+SpF8lSL5KkXyVIvkqRfJUi+SpF8lSL5
KkXyVIvkqRfJUi+SpF8lSL5KkXyVIvkqRfJUi+SpF8lSL5KkXyVIvkqRfJUi+SpF
7JKsXA9h9EVMh7BfXVpVIvkqRfJUgiQo5MA6AHLlNf834jK46NTrxjflXxO2ojG/
SNEY36RojG/SNEY36RojG1cYcPVSq2eXyVIP2trvzwdTYKt6KYsW/7kspGF4mFMK
XQ2+KDMmAl+NBeNhNAqL/CMLGns/8q/8aVSL5KkXyVI8RlzXcLkzy+SpB+1SiT5n
sqkXyVIvkqRfH6Z/69x0a5YuYKyqwjUK8tUyj1qO+Z/L5dfC3LMW8w3lDRcz9u3r
PlqxEc0Cov8IwwNcOJ5ihv3zxTpG4mlQaJc8KH6HVJet4FReJwRSwhBERaH7ou3a
pKK3a2ZzsdWo12zy+SpB+1SiTZHk728u9jMZT4rDe7Ul/w2uEDumFGLOD6xRTvuv
xMhZsXVbIrws5K46NTrlhhWlBUi+SpF8lSL5Kg1PnXjHsqg1LpIXQovkqRfJUi+S
pF7VeFnJXHRqdcsMK0oKkXyVIvkqRfJUGp868Y9lUGpdJC2kHwZsHSuRYzH9Gxyv
p20a2GZSqSK5f3bO0rfQXxpTmxSvE5GPW8CovE4IpYQgiIs8vapIDLST/oIELgxH
7nAwDXV/h7VKJNkb/IhDAflLOhe+A648YeTCd6pgvHeZy5PMcywtuINKr1HwiB9N
wRzPot8eBURzIxTvZnQgcMbCbZ/VOvl61msY+3rQ6qVujU65YYVpQVIvkqRfJUi+
SoNT514x7KoNS6SF0KL5KkXyVIvkqRe1XhZyVx0anXLDCrbbFThqflxoL0NA8Vx2
4yEqZJkHxZgVvQoQGRfJUGpdJC20N5GgnzaPwlWuH/BjTPDKpWHE4cRXObx7VeFn
JXHRqdcsMKs7dxRsc3kQSV4xyWJEWeWrERzQKi/wjDA1w2WCXabpKe+dUmJAebwH
Ws+xiAsb4EiUXBFCLJqjOxNyiQHx3bbnKEH7nAwDXV/h7VKJPpMIPkqWMrdGur/j
Irws5K46NTrlhhWlBUi+SpF8lSL5Kg1PnXjHsqg1KNU1lIfFBPr4Ck9poX+NKpF8
lSL5C6iJ+mqmG5HnL9iq8Y9lUGp/VABX06g03VHEGm6o4g03VHEGm6o4g03VFtHl
eU/qnNAqL/A4DzmiiMb9I+4L/GlMU4M7AYBrq/4GV7ybFFvUURjfoCBASaVujVBS
w03VG7oegsjN3fX5W6AAa0GFwjQwmcF2yBpIh8EjK4sP+JEzaYHxc2Baj0O4jXzw
Vj01U/We4FH8mUv1akFXNdqjQFMdca16zF4LrIVBW0+SKsVuxDLhs2Q1L2xlwEUb
KkLxeodmTx4kvd6L+ZBRxBpuo9mk4FRf40qgJeW1YxRw1s5M66BfsUwHOfgYGt5a
yTFIDHUkhUkIKgKoy2TAnSoZH8sbcd2MMARI/Mrb0HUe4SWc8poEQFdGfIAFLk3Y
C9LXhBK697xsECmQtB/cWb4LuQC8vkqRfJUi+SpF8lSBdlr8lAEzzNZRgCofQ6c6
8tnUbezPZVIvkqRfJUi+SpF8lSL5KkXyVIvkqRfHYAD+/9UGAAAAAAAAAAAADGxC
JsnP/b3GuUc8ro9WF4GFtat8erNXoBFQ7KeW3q9aQ8MwFhjB5jwRHCSR18McUPiq
cvNIkd0E4TB2/oBlM4pyK1Yl1w+jE7nTN1skmb+bVuQsC5obJ01dZ0a+xBsnmhEA
CV5t4CwUSRlM4hFu5pxKXaPb9cyQTaF2c0qopzuMUsOHhN731BwflmFTyVbyVVyd
QsgGpezGO4UMxvzxZsV9B4fOP/qqhP0p8+NKuqRCMouoClo3bEgKXQEOgF3GVsL7
hY0Xuyg36a14HitJyMIF5Ol4mIAHdfu2w1aXb3q3yHGpxh9lsbETry6ESikoBzh9
jRmo2TligyziJ15dCJRSUA5wuVXuN0TdZMHH4mFxdflEGY2zmz1FwvSNM1esenZN
D+mfD7iXug5hz/8bpBXtU7hNuUYrI6AjmQUoZ+ubuJtCeUFpKBCjvWIw6eqxO3FR
uKg4BREzgAAAANKTXJ01rKhL2ZOAm1nJkd4UrfCbxp4T4sM0BkjVkuCBK/4e2kb4
x6UDmyg3YP3zjINd+Lo1dhcRRqVT4hGIRcAr20hiOm9mkm1wvonvmYs7+xKlvcaH
VnMDPu4AAAL4fQV2yxu3roqfCKwEUqo4BHwWaKQglzTpAAACkAkVdEMcnbYRGmCj
fu9t3BAZCHOz/cKO7AiusnnHMlIwm7SPKTyDHqKtlxeO75elJh98W0nH/eUYmzjV
c8Az+/xjxfRE70wtSz8gmYeLDKx5liBZUPk+XaLmrdjJVK18bAJwNZZqyS0E9iwW
RvntNkr98k8mqR0A2YiC8D2tTYvZiBzbMGCbWqb2sc8hYCUtyQjCoY9z2jGlCuGP
Hq1PyNqQWg2faYJpi30nrgZwHxeeDvDggDAsMRs2/hEtVkUhJYxw1I9OF6X0EEKS
G58napY0UGg87fqiu+XMZmwuWE9x+0hh5qFnv6e/C/J9RwA5K2dRFCQiaJR0eGzg
xIHIl0RN+4aUjh+bDVu/ltKgG64ehIFFwvswH0AejSQ0IHxyWUBANp2XHwSgmC9O
f04Xf8pkueCriZTbuRjwSigGis0TQN19UU9emPqXKN0TZ8c6ExJe89sY3hG+30oo
NjmllEs9WfsNg63Vciue29x330DmO9hxssNQd2t5BE0oo6BKWafaypVFIdEIkMDd
ESJ8c8pYf6EsbfbdioGMGRttjc+PVn7DYOt1XBvC0UxCnwXE/mxuAmsozpWXAQSr
SloNlWtiDY5f/0kurrNwuB3RQRtB0uT0UthufY8XxesHImELEYZ+iPMNoVu9FUFu
UDT2OAIlrwOAAMUwBYHQ7wdtCrjVdVDVl+SjcD3aymtpPOEEWcfPuN9Maeb5cZga
esCw8Cz5fgXtrC/xjUF8qdzyk6OPJkI5NWZ+2P9o4a2ba9oSyGPsG+ZTVe5+KyeT
Fq1ly954jwEtIYCJfoYfu3o9L1x+/PXld9LfNnluqHuCdgZjeEUK9zndFm2mKPnj
fXW/FfXzDiKxTOGyr76xubA2QX1gRxwfdTGrPdTGrPUepSNN1pcgTdYYkne6BR1Y
ehLcmoNHIEOTiLMkJTk+DxteJKt42gex8lo8ZTtxaoQfOpXoWEcg42mFFu7QdCQK
oJ5uYK8PfT28Xwmgd0VV62VVVlJDQsXtubI/Oxg8t8/W9XIZ94DVE67WBLdXpU9b
J7csxqy79hC7PYfyVNUXDMGrFuw5VC/5KMSATRazarHd/BNSZfaKS+2UI2jGpDgc
AOD6JV+J6onN+ATp2PYC+0P+2oSId/WCIu9rVIiaLqTOw8Asd2f3lzNgxwevLGtx
7Ta4+rlAiRWMdWPDgORgBRQ3SlL5oubBeEm1/kU58peqlA1jaXHndn+YI5DTBiFM
qBXCuYm1Q0cLVOsp1xRIrGOrHiSbFZKEjK7IX45Zoa1ATmGU3xWW5UduVs3k7vpR
N3XJiq4443JgtHFLmnl4UrKBZg46iIe/4qLccjACeC7BvKyKmigAAAADRMAAPA4B
onQ7wVOT32P2Q9D2mw7lhEeR0hsyqvCHIfJMq0GZj8yaQTV5fb2vsX3H1zEH5D6C
wRZlOmVkXAlJKeXG5g70D/hIvzgXnjodFGrby4Hk5WV0Cvzktp0qlfXzVaes1scY
9+xxDNUWcgQBk5GWuNi7gHnflWBcE0jvORfHFKBdkbDchDdjZRMIxETd4sBZ58V/
uOpGisHjENDxGkAi70ocThyfelMcpZaMEgtx3Gp3fxFYFcuFb4smrRQiZLUR++w+
FCd2/Usr0vS9mtd9RvjLablO7J+qB4TN6K4nIqLw9G3u8VcOAxDpofL6lc+pRFjj
RiixxoiEqPSDLgeJQ70ckjy/79YIbomyzZIFSZoEP3EX62KYBK+BaJbbgV/Cb7jD
GDa8EZiJi8zpt7iHX1th9jCcj/+s9CZmgUWM3dd13XGRrBixmI3LCT63Yip/Ug5V
kQjXtwF6khTeWIjKyxvxlZY5SHQ7wXUjj4nEWV8VV6t9X/t/8BkfHkd1QSYhmX8X
G6s5X3oebS/OIJXm6f6oDdSwRAH3XkPoDUfNr32GIkfvNeBY0SETtzmoDOKF6k9k
L+Xk9VyUw7373wcn1ifYLEcZDIWmtxg+PfgwMDPMYGBn1O9de9VwqL8j3w9VxQ2f
CM+hYZCxsv5rp2x8mgSAr/3n1Ne229dwWjzuZZp4H/YrhaIa6AhmO85ZUiMo3K7w
mE5MCk/pIo55zi6FBi5YnR6cAX+7oQHaJc80A/CuMAAAACzMAWB0O4S2HXGjq2Zb
IhEYk2WpipLfJFvgL+y0zk7RG0/MGYOpmDMGdQHXYEY1Z7MFgLMCHn60dXdkd037
W1QondEqqeXuyf/lG9lGwGIdIBbRqbB/dIFvJ9JsI5BxtMKLd2g6EgVQaE8rrQIb
FPZb71RvPR8qDmQpo85w0NiEaWBaDANZ6A8OhxuEWNcWcPjhPmxa74+agR+9NivU
EQim1NmZN7iIKwJ9GldFaSH6+lcjgIW82u8ZdHyL6S2Uyvy5RADtM+iwVC19CZ9f
QmfX0Jn19CZ9fQmfX0JcZAnpUGwGkQ//d9asMiBWJqJfQ+A1KvNpqdk2N8gyOsGR
rxhZV0oNUdR+ftfP/09XvJ3M5/vznDfP4uZt1oYCMjTOoX1QTIEzOlxMOKgEGUTB
rX5oWyxhtYu+EOO93zuZN/bWRi5MKfdAO+P91fGpaofLNZi6a6DZdzMlmSCWyg+h
b+sucxgYOtt4Fmi+djAcmZc1Axe+zFoFY8u9NN9rt5PjcTIiMCIwIUaJTz4UM6Q+
k02zThgFXghM8IAB29ej4MqhBJ9zJZPY3xZW4gGTfUJmhYt5RMnpXJIsaa9LOvDi
kz/2Kgo3af520f8Xn5yjGIRmOX6xkUTZU+We6lw+dLWe1Q6I2leK1RUgHXnipLMh
kgiHnOPItjERWfnlTLNvfiHx4Xwj6iTWGyr+lqc/u8bl+6GMo6+QYmgeG6sxWSod
NPohFABMOEWibHyK/FNax0mtAhf+yjL0OWN5uAzd2XTejs40TcpSEdEwiX+MCFIe
NmWzx7RTQYDHQjFrrXw8F8gjWKYP+sHabNqTlGD5RmvRigmjoeWrOntkd3iQEbgX
3F4RiNouLzeSCmav/FiCU6cDoVJ0u1cFQg9x+YmrgUt5tdFBxOYE9toAemj2ALkS
58PZl16MmGcK0KjnZ4efBBgiHJp4fh5bFp5rj0MnbKrZ3pjAqc/dGn4gBx94UHHe
lwUU9ae6kMbbT5EL8vI4o1vkrfKwao6OBfQqlszfa9Bv4ugOQbJz0w1Ead8ALImy
5d8BtOaRCDyyF1RevclT6L0o8Hb3ufvhbNn83fUauGK6IS53nSkFie3wBImEuPXW
g8m8Bqlgy8IMEaxIvpLDoajJYjplWxyveJS4cKOOSFVSd1+kkPG+oer8yK8Cbvdt
oMfVE4PfSUB9/1YNPS3A1Bp6ohyKW5IsnIX+xJ5Jp3JllzHEOZXEe4u0lnC3tuMM
nquQSLW5CdVswR1JsTPdCOoLbDqYUW62mitHk8Mh4KAMQLdFeDZ1XO5fUuZ5cZEP
eINANQ3HuiU+LiJBElZgRwapeX2BlBOvEi8fDhtnhxb3uciozEXtN8+DDj7qHnqs
wkQju/3e9/SvTgtzdf5/MQzvDqpb1YPB21gdWWOtJQSJSrJV/c+O0GWtiQlXMysT
k+GYwR9DivAkdJE6j91nPvNBFy5DGRuT+SVzSuyxqXyneaRwFTQFdzX2OstqJuMF
MmHZs1QdclgMO4HJcFUU8KUfbVy1N4/O/+tuC/kCL5G3DShxEn1VbGyiiJ4EjWUP
s1sEcMfO3YUxvyLXq7SQ8M1RkSP1wYghG50bvxEFwiq9/S5l4Lx6Q/ZjYOfXHLWS
HxzrxsJmxm8DqJCZWnTtgznPkEnUKdGswbn3dx2F7iAVGAUN1jYxLZtH4w0xpZMT
yeDPS6W2qp5Q25CjPk1Vx1MB5HC2usOmpHfAQMcFhXG7C0WSajtlUDgFoDSG8svn
qieOIzDRX2VVv44AzjIE5P9SvLdTRk4Id9zFp53STRrCqn2WbaHJd29nZe50Golm
uJoTTQvA3TisjS02gJ0O4gQPT1+aQGM73BTWCFS5nngdg8k+fpvX+5uxvhcfYFM4
unQ+upfnzc5Z2g9MSoIPnG8h2U2gsptAXG5oWpnARDxv+kb7/3vcD5vtpbKhFEAF
QTOGJ+1sN0+zdK6/MOJMZlZLz//bIQQWGVqU5SjqGk24sySu+/3Nad5uwocNTtZo
PPBlgjF5BccFAbVt+rw3oqTZxIxc4Yn7w5x+bN9/LE8cH/VyUY7DaukySZhw9Mc8
FNU3c3anSP/Dt6z/M2jvlLzxZgtaZg+ls+1Upz3FlHGwM32WOiz7A0fAzumxuZ0V
QvGr1WFLLovK/PTtyfgFC+XXLS9XREFvkw4fwkKpE+MoAn+8/UvcmWOlKnTnE4tH
TF08+l7j70LZKVg4Zi78XJo6Qx92XVspTNmhU0xtjBlxxaH2i+cNG82CQCQou0fA
+Qo1HDrwc86IAr88K3hLbgKkqBlxbqH6VJmxe9QhIbM6GfD1zDi46IHWAfztn8bu
htj3WgLHLuNcuporLQKv4WgX/BARaOOBLrBnZTYLdKmqX8Pksna2O3dH4AeBHSTc
5isLeCu3mrFekDXjmJqXD5lEo5QVEwZJzJTfxpCM84PpH8pC4Ykf0oSVBe3YTZeL
E3A+N6z2xS0Hr0YgvinjvnVpdQ9P9txfEjmyrMiW4TY0prERXwbVedwuwjr8gS/Z
aFYjcrq0fJXUgAcw5GGUFhtE5HCVUrmz8r/6Xr/8UixUD453u9zCZJgwVE99Y5eM
fsxI1cTO4H6NgTPLlYKu6UOqI9rIR5pCjyYXPXT4kvhOGncEVZOjpjaDCkIhf9Nn
itq7iPzNvdYXmbeXURsYTp1dskG1vZR8QtQIC5wuA66yTOlgUqXl3BBHOy9sAB08
Q2ID+MYZc6H5f9/l8Z2T1ANpdvya88WlwpAfduVQC9ZOQMiLs4RCeWu+C0z90epl
4ifE4oS5i/cSL/ysyw9omz8shHVOFxyQvdz9v0Ee7S6cXAYAiEYiNSxRiEU7UNiN
XuXvU3rKDyOMSMDkZwv+U42cn2nusujlGiYtt6s+FlCRdbSpIgxBI00FFuNlufYi
2u3NxGnQ7Um3k4CI+0btAOOiIopBdDXEXk1nnHJoaIstS/yYT0A/gQz7ZOzzef7Z
4AAA8SbCvNGgD1HWFgN+amAn5onfnGEa+nB2hxMb5VAuMAuu2ZuzuEX3rP+tYBrS
LqmOeeQK4DUexhiqXILqEYke3sbpWTz8z7aJXzmgyxBtZ0a/c2L9G2uzJ5PqMvEy
ALpvaxlKeSOsmNBz4nTbQXoDJBvVS0AEX8W7W1aQL57x38vN7tbz5R7Ix63S+I7d
ngtCYRUjyZNkVxPkHQgDOziZaTP3B0r6AWxaS7qatUx4pcwRgnP6X0oeNJUKRDNG
bF/s5iGGEZ1Pvu0EnsniizhnxkHsMG/j9VZKgqIGhA/OGsZm7gFivLB7RWNqmlhj
50NHS6Sd9MiPQ24cu46NaX04AK/+CBH5bnFshdzb86zTmVotL56iMgAqMv5uJBy6
4rP9hzPZvQPlEZjdw89QZehBJLfS+RnjNbWBBSlliDU2Q5nlzcj2sG53+dY/33FE
GwdH2nJHqWqo6YglcSOTOqdPc29pADeGhbEFIQFBjes9NKAK+exH1GwTq0awvX/+
Pv7H+nCauwEGXt39FUJ9fftyRJv5to/A0S/V/SccVOXnm8+Qd7M09mYfi7fY5qfa
kQTs628yPxuhnKACWB4734sM5/eyNGMZ1suCRgNLVYORZgM7Oqz+k+/g3wp+fJoR
uKroXiLAmW62A4jG1kMP/cQpz91GQAo4mQvgwmLiwTvtnFXtk//ZFxLo0Ds37eHw
jwXJLojLVPgeuIf75DVOKz2Xb1ajYi1FU2ecbODCF8n8xk4Aw9eH/sCk12D9RF/Q
B1IdalcyVryxYRk42HIiW40wZHXF8E3UrXToyxRqg1IGfItGVcqaGmHWeuIw/coe
3TkXs3c8M6nRp2blp2XkDVX3QzigpJb18bLX3A8vqxezH1LeDY62gg17za+ClhP/
FLRfobmFYJXYCwBi6y7/zZY4+NQEc1qsU1JAXMSfRDw+5jQa5F0XuKTs7y26s++I
+lQbnTMgZiEAdM4fi5rq9JBTlU2wwhyvwMkWoViXB6UTWOupjTGhsbZ78Hwva9IF
My/HiECylZwDdiscQ5J6pbG8URvHkmLUbIufEuGmBL6Njh8j7DcoyOowRixETWqv
AS55yhFn0CqOLyK/x2TYp9zXn9VWUQJMYl4l5NRodslkFubRoA4msxdfekJcqDLH
sRKIkDkFw+0HeKHEBdkgpG0ExQFW8g0brIgrUcjZTkqg8Unj1+S7qTR8xuIRRAvc
s3DWYztgp9UKWYNBmEo/9XOd22y1Mw6JZURokOoQM7VKezi6BEIhQc6yAOaStUY6
dHxu0fyHgtWhc402pHMbY/s5/j18TWPpizjbw7veeI6b0dk+PKh0jRY+V5BG1D82
A5N1wVwNQRX2uxQ1KT/lmLrliUk8KOZZz085NYp1OBrOoBvFA+g4DQo8EH94zhZF
wxUtywJ0OXh55qWfrmXYddK8ckyVTiPBXF1IdR2tKzRD28IfyCrewK3VupQuii+0
FVsEJ1FBjazNpG0fBw+nnMblutpJRMWSDPnmdVZ9gJ/UK7OxjCFeCmrjnknPd6C7
mAJqzuFPxzVABOF2nS6/5iUSFxi1moa9VExeVFL90dablGu2/bvJ+f2QbTBYw7vS
kwCyhr4wUH2tKkPLA6HHTsZMA3NMmgILMaTotNEmVa533Fv5kQt3/y6inntimw2D
cWfeIdgq149uJnZ7w6mV+vkil6b00eLgVUX8PnmcDU0fjXZc3hGcY34Vh4cTaykF
ZmD7oNpO1J5EMR0JgICTF+kIMKxU47Z1qyfnlA61c67FJ7IWZadDjvp+eMBIeTA7
KWU7nVFnmZ384Vx/94j8Fj/Hrr7rk4m/3RKkZTUW8bGnVGe3/8MoKEqSNek9lBFR
+bjw4XK8gWyQ3CFRDhKgzdlwwDzRi+P9sI46W3fPpeJ/6zs2I0JoCinvGUAs01O9
fESdg6XvQTTfL8n5r2m5FLHcPokFwNYEZMfKr0gAAC6Oxs1v8ftnbm2EiazJCR31
OStFxispr06BEV84Gx9gAAAAAAE9VcyGOGaZryWDT6GNTztozjH71dnXTIE4dSiw
JMq7ov4Zm32rhTNOySnXwXwmdSB8tskCavjZ4XSO+pm+WdUfvrR/6AWPp0xd+MMi
XCdmSFa6pOmTMxnAAAAAAAAAWE1QIFgDAAA8eDp4bXBtZXRhIHhtbG5zOng9ImFk
b2JlOm5zOm1ldGEvIiB4OnhtcHRrPSJYTVAgQ29yZSA2LjAuMCI+CiAgIDxyZGY6
UkRGIHhtbG5zOnJkZj0iaHR0cDovL3d3dy53My5vcmcvMTk5OS8wMi8yMi1yZGYt
c3ludGF4LW5zIyI+CiAgICAgIDxyZGY6RGVzY3JpcHRpb24gcmRmOmFib3V0PSIi
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp0aWZmPSJodHRwOi8vbnMuYWRvYmUu
Y29tL3RpZmYvMS4wLyIKICAgICAgICAgICAgeG1sbnM6eG1wPSJodHRwOi8vbnMu
YWRvYmUuY29tL3hhcC8xLjAvIj4KICAgICAgICAgPGV4aWY6UGl4ZWxZRGltZW5z
aW9uPjU1NzwvZXhpZjpQaXhlbFlEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBp
eGVsWERpbWVuc2lvbj42NDk8L2V4aWY6UGl4ZWxYRGltZW5zaW9uPgogICAgICAg
ICA8dGlmZjpSZXNvbHV0aW9uVW5pdD4yPC90aWZmOlJlc29sdXRpb25Vbml0Pgog
ICAgICAgICA8dGlmZjpYUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WFJl
c29sdXRpb24+CiAgICAgICAgIDx0aWZmOk9yaWVudGF0aW9uPjE8L3RpZmY6T3Jp
ZW50YXRpb24+CiAgICAgICAgIDx0aWZmOllSZXNvbHV0aW9uPjcyMDAwMC8xMDAw
MDwvdGlmZjpZUmVzb2x1dGlvbj4KICAgICAgICAgPHhtcDpNZXRhZGF0YURhdGU+
MjAyMi0xMS0wOVQxNDowMzo0MCsxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4KICAg
ICAgICAgPHhtcDpDcmVhdG9yVG9vbD5QaXhlbG1hdG9yIFBybyAzLjAuMTwveG1w
OkNyZWF0b3JUb29sPgogICAgICA8L3JkZjpEZXNjcmlwdGlvbj4KICAgPC9yZGY6
UkRGPgo8L3g6eG1wbWV0YT4K
" />

In [11]:
# ⚔️ Runnable Code
data = np.random.randint(low=1, high=7, size=(5,5))
data

array([[2, 2, 3, 1, 3],
       [6, 2, 5, 1, 6],
       [4, 2, 3, 5, 1],
       [4, 3, 1, 4, 4],
       [2, 3, 1, 3, 5]])

#### Convert to a Syft Tensor

We can make it into a `syft` Tensor.

<img width="500" src="data:image/webp;base64,
...
ICA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgo=
" />

In [12]:
# ⚔️ Runnable Code
tensor = sy.Tensor(data)
tensor

Tensor(child=[[2 2 3 1 3]
 [6 2 5 1 6]
 [4 2 3 5 1]
 [4 3 1 4 4]
 [2 3 1 3 5]])

There is nothing super interesting going on here just yet, but the purpose of this `syft` Tensor type is to provide a consistent and general interface for operations in `syft` between different Tensor types.

We can see that the `sy.Tensor` has a special attribute called `child` like the diagram above where the original `np.ndarray` is referenced.

In [17]:
# ⚔️ Runnable Code
assert (tensor.child == data).all()

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### PhiTensor

**Note:** You may see `code` and examples that refer to either `min_val` and `max_val` or `lower_bound` and `upper_bound`, these are the same thing just different variable names.

We can then turn a `sy.Tensor` into a specific type of Tensor which can help us with DP calculations, by annotating it with special metadata.

In [14]:
# ⚔️ Runnable Code
private_tensor = tensor.private(min_val=0, max_val=10, data_subjects="Ishan")
private_tensor

Tensor(child=PhiTensor(child=[[2 2 3 1 3]
 [6 2 5 1 6]
 [4 2 3 5 1]
 [4 3 1 4 4]
 [2 3 1 3 5]], min_vals=<lazyrepeatarray data: [0] -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: [10] -> shape: (5, 5)>))

You can see from the output that we are building up a nested data structure of Tensor types inside other Tensor types.

In this case by supplying only `1` Data Subject, we are implying that every single `scalar` in this entire Tensor belongs to the same `Data Subject`, and therefore by extension we also have only `1 unique` data subject `per data point` and therefore we should expect this to be a `PhiTensor`.

We could have also passed in a numpy array of `Ishan`'s that matches the same shape with `data_subjects=np.broadcast_to(np.array(["Ishan"]), data.shape)` and get the same result.

#### Tensor Chain and .child

As a general rule, `syft` Tensors have a `child` attribute which returns the next level down in their nested structure. Usually the final level will be a `numpy` array or some other raw array or tensor object.

Let's take a look at the type of each `child` one level down at a time, until we hit the bottom object, which is no longer a Syft Tensor and therefore won't have a `child` attribute.

In [15]:
# ⚔️ Runnable Code
print(private_tensor)

print(type(private_tensor))
print(type(private_tensor.child))
print(type(private_tensor.child.child))

Tensor(child=PhiTensor(child=[[2 2 3 1 3]
 [6 2 5 1 6]
 [4 2 3 5 1]
 [4 3 1 4 4]
 [2 3 1 3 5]], min_vals=<lazyrepeatarray data: [0] -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: [10] -> shape: (5, 5)>))
<class 'syft.core.tensor.tensor.Tensor'>
<class 'syft.core.tensor.autodp.phi_tensor.PhiTensor'>
<class 'numpy.ndarray'>


The top most level is a `syft.Tensor` which is quite general and does not enforce any DP metadata.

However if we look closer at the next level down which is a `PhiTensor`, we can see there are several extra attributes that are part of what makes a `PhiTensor` special.

<img width="500" src="data:image/webp;base64,
...
ICA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgo=
" />

**Note**: We have not included the outer `sy.Tensor` layer in the diagram to save on space.

### Upper and Lower Bounds

If you look at the diagram you can see that the `PhiTensor` contains two attributes, `lower_bound` (or `min_val`) and `upper_bound` (or `max_val`).

Just like with passing in the string `"Ishan"` instead of a numpy array of `Ishan`'s we usually pass in a single `scalar` value for the bounds because we are defining the lowest and highest possible values for this type of data in it's real world distribution.

**Note** it's not simple enough to just pick the `min` and `max` from the actual data because this is likely to be narrower than the true distribution and reveal something about the smallest and largest numbers. For example if you chose `min` and `max` from an `age` array you might get `21` and `55` when in reality human ages are positive only from `0` through to `~122` (currently, according to Wikipedia). It is therefore important that the `Data Owner` who knows the dataset and its sources best, provides this information so that the system generates accurate noise.

Let's see what the `PhiTensor` stores the `bounds` as internally.

In [18]:
# ⚔️ Runnable Code
private_tensor.child.min_vals

<lazyrepeatarray data: [0] -> shape: (5, 5)>

#### LazyRepeatArray

What exactly is a `LazyRepeatArray`? It is largely in the name. Think of this `type` as the inversion of `broadcasting` in `numpy`. Where duplicate data is known to exist on an `axis` we do not store it, and instead retain the desired `shape` for future operations.

<img width="500" src="data:image/webp;base64,
UklGRoQtAABXRUJQVlA4WAoAAAA0AAAAiAIALAIASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBImgAAAAFnoKiRJDW5DPdiMHASLiICxMuv/a2194Wb
2LadCuiSDASgCR1IyRpoaTOICz+siP5PQFAe191b5UFxXvfvFI3XHRzPt1C23UMX
//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef//Ef
//Ef//Ef//Ef//Ef//Hfrzv+O+6h5R4abqGkOG+gOgRWUDggFB0AALDCAJ0BKokC
LQI+MRiKRCIhoRBbJMQgAwSyt3JXt5xL0G8J3/OvyV/oH/04qncf59/Tv1k/pn/X
/6fVNb6+DPaN5lRR+m773/Uv3I/vX//+E/9l/of9C/wX9V////w/Iv5k/tfuAfo1
/fv6Z/j//F/jfhj/UD3Y/0z0AfxL+hf6r/G/vx8vn96/1/+m9xP7B/q9/qvkA/j3
87++fvQP7d/xfYH/nH9y/6/rO/+X/C/v/9G37N/93/C/8z///Qr/Rv61/z/z//6P
0Af//1AP/R1y/Un+r9rf9F/HjrHvAPr1+QXxKfo3j4iR/E/rX9o/qv7c/lV8gf17
wx+Gn6/+OnwBfiv8c/sv5G/kpyGYAPx7+W/3T8o/7P6Xf+L6CflX9x/t/uAfx3+X
f6D1K/wnig+cewF/IP69/yf8R7rH8T/x/8R/l/2+9t35b/fv+x/jfgG/lP9O/2v9
6/ynvcevb9qvZI/YURUWzAHdP2/0FRbMAd0/b/QVFswB3T9v9BUWzAHdP2/0FRbM
Ad0/b/QVFswB3T9v9BUWzAHdP2/0FRbMAd0/b/QVFswB3T9v9BUWzAHdP28jfZQd
AEt7GCc9B0ybf6CotmAO6fkpc0BhPYOnQA9GOm5k6UqfjUSRe1sFazkUN3nTlMmo
SL8ZWYOnQBLex+7zpyjMhCy9AFBUWyWBSNGimcaYjXz1SY0qi2YA7py0MTNIDzvd
eErctuNtPuqZsp7uoG6ft/oKi2YA7p/ABg6c3M9yF0/b+Znku5sRbMAd0/b/QVFs
vQCGdjDun7MW8OA9L51PhUkkNAWGkZpddAPNmr7EHA0Ufi9ieDGK5VLZi5nCV6pM
ZUhtdTpQIvWwTHXfmI3BV/3RNkKS2YuZwleqTGVIbXU6UC/yRSN5wWhNtCCLiTQP
mJFFLmxFsvQBdE1AtMlqPqcCC/Md8hJN7Y50FtxECrv6belFgrZA6PoRHrEwC2CO
mRK9UmMqQ2up0Ad0/b/QVFswB3ThR2IXyf6Cdms76DZgDun7f6CotmAOwR0yJXqk
xlSG11OgDun7f6CotmAO6cKOxC+T/QTs1nfQV1a0lyMglsrfRBwsJmNB6KIrIZDa
n8pvcHK0VHtFwVhdquo4UdiF8n+gnZrO+g6RJOmUXP1xZC6gi2ZtRTs2xkabZgDL
go+SwD4GDk3v2uhVNE7FozyK5PHVgbQNer/c4+Tzj1BKPdk39dtgxU+g2YA7BGxQ
aoUeTr3N5M3RBmt1c8Ri5nCV6pMZUhtdToA7p+3+gqLZgDunCjsQvk/0E7NZ30Gz
AHdP2/0FRbMAdgjpkSvVJjKkNrqcZKvdzxb/kkdiFXu54t95Ww9kBVTe69oYkUUu
bEWy9AF0TUWhkWF0he7GqYwc+iuWhlsnEfODzSkwwg4xU+g2YA7BGxQapMyBIpG8
4LQm2hAmJCzqHZtjI02zAGXBR8lZCThq4waXUxPSzbsTSHjqwlyPg5ofKJ3fP+f1
nRN4wMpd6EAgvFKguArIXT9v5meS7mxFswB3T9v9BVCXESXM4SvVJjKkNrqdAHdP
2/0FRbMAd04UdiF8n+gnZnvSKU+uKe4O9FZmX/QVFswB3T9vdODwASs1DGZLBj8u
bEWy9AFBNdzVeMrMHToAlvY/d505TJqEi/GVmDpzgMQ3jes/YypC6ft/F0+csdg6
dADT9m71WR0Bp4yhMdCQUzF518mv4AsVuUo9eNM4bqj3bQiTTZrTzK73B8lOn4rk
2tIMfBbo9rs9Tf+lJuLGq2CWFQd653UmUxWUWqcCSNYPJ7hn3fVLhWEadO5ldBYJ
NKIAllDqy7e3VjODNUmYrdWdT9yG6WRhNy77kh8OBe8M9AP54PanB53gX91qDP3/
t59XBd9xfe7EEjYRW2A72iIHGNI81u1SNkqiAhgD3KaKrVxIGEZr34xCzAF/+oIw
XTrTfqfTHYEz3x1lVM97XI7SeRCR4/SE9lTBJDdguec3s6NxpBAznSMKEB4RKwcd
dVkxpVFswB3MTnVujFq+Cxj7ndVU4Iag0NP2/0FRbMAd0/b/QVFswB3T9v9BUWzA
HdP2/0FRbMAd0/b/QVFswB3T9v8/wAD+/+zOAAAAAAAAAAAAP2/5Wl4oNkIT3HD8
I/Dpiilcdd26Fvgy444UeqbTrol3L+00/hOuqHYICEN0lcpbNQpWcObAv0gf1Ux0
K+IV3Qzvk3Iaec+BIoj+nByx5/gnmEQLXpnkryCcDD/Lm/fYlz06ht0zMO4lebik
IApDCHKwY9A09rll7e7Z0Rv5cfImAHuP2jqxj+5VDXccDNOs1ftNP+1O1i3xxJcu
g5uF+XOvWPb6QP6qY6FfEK7oadJdiPv4QLjbi+5BfyFnjMnz/1gyNtfixbmMLM2W
BlMTRuZwFE0wV0lN5D4gX/wZoEJiZQze5G9QQyY6qOJ+RBzrzrnvP/qzLidOFXdv
IoVmS1Sgc7y4MpMTxsEnjq8+hk0PpYpPc/Oa5giyZmt+STTdk6JgEeW/mOa+jVOZ
GfMS6AbeqjbWq+RGliCne1e6pe6+P5Ef0TicTicTWUjX4VE5jbKDAyy5gpPWg5pt
qijsSR0CYEMGGC2r/m/kuw4lwO1ROpBUynAXMVVu9GECMf75500gp4As84dr4EXk
IYEazSeDT8AAGodmT87dgaYgUAfw4d2k0AAWKUJoFMcAUhE3Is2ztb/RDK/rizwb
ciLAHV/1wTBZpzbQCyrUDespQLvQGWcjbJk+56KEsD8/TXD/KnzpPWVVRAdu0bCB
sKElPfdpk8nALri2+NwEQ9x4e+iv6QhIoECidwuRrvt0QyUIXdEo4B55zmzlm8lC
inM1N4T0M0feiKKCMFPs4V7YDWRJVKo/BsZJzqPNeF7Qa1hrhBgrhnH4pfhgrfqN
g5zrDkeTVAvF5XPLhfgMxeTMZboDxWDmICuhxLzKC9Kzd0YkWCif1vwoDlP9alJY
55w1N7XFfdYYxkKxvsr69LF/HKayhYcSm/Z58Bq3ke9t+7apynEivJxjp3J/ADHD
lcJtVj0qB+OoYY3XE6ENb/c5DwK6iia2LMVAgvACyDxYfQXCbjDAMLq+prBaLb2t
FKVe3aFzjRwna7+bTcfpVwHo1fuOyDyiG+h3PMAAAANY4ACEKAASazZsebmJf2RB
7Ch5XrIpp1hvmx+ATUS7kHbYYhW1lQvDEjkZ9LeueRjtf/jLi4LgQG5qk1gr2/E+
3xiLO6/PZshUOaWsSrzSD2FCS26JMe1CSoTd6DkAUIh+UeJB22GIVtZULwxI5GfS
3rnkY7XxorMXUBxy8JAXB2L79YLJ15SqsgQFUKiF3PCP5Pj6mhMP9Y4BN1sudDTy
hRqsfaBdGccnxNYvHiOcaKU/AYJWJCMEYtgngpenwCo1fBVry0UZLN0wkRSAAAPN
fFKAdh+DlJ0MW+jG9LA9eJ0fuM3hRvhoDIJiPcyMK4W6Vr+0aKem00UrDwJvMah0
FOmapTOWVJIOHDHImADcsAQiYhrAMYp/Xlbev6+dkiOJxCc0IMsaDToX+qSs/o2w
+ZrC/W/+C22tltTzK+Xw3DnOQ/tS3sfH7JwGJmkNBwO7Lrcv3tkwMg095dbRK1IW
2yFyUAAAAAAAkygAFeYAAsE7NfCBeHjkzTOeIUdP9DAb12PlU5j7geSkFpLkW/5G
RvTARulgSoCWnv5ihCYKE89Dty9OXvJi06C6lJ+k15H7UG/VZLs21AL+AGqeifll
N8KWQplHunP9Rq94HCVl2HIHn4yxfS/JOUPZDaZUuTBs5PRcNpuFHcAvKQEW+hE6
NCmUe5jDEgTYo9xlgXq85qIPSlEoca4QYQlQYYTT/JvhFZZsNhXNn5tW2pHvueLP
FnizxWJnQ8Apy345iQ/gsBzXOUHpXDNMMOgyMCHO1XHeJOQKCQkqkxR/yl0zJmTM
nMsjfaqcJyXqsQEvMl4Or95uzMxjfL1BgSJwCpOAJYbhs4mZuJ2jdPaIaYiEG2h8
jZ19BveBkvzgRIC6i707iFMNEAfFmmbcE36psbhsyTsXq9untENWyli1cfVl8zSu
4/XvW4MSV9LAYC0/pk4RQhLNFhJk0mBoLOn+tbCBt3YpoWrNQ2dfnsELTk3hQwh1
dtfycpycdasUsG6n7OeN65a+P3DegNY4AAACRKAAYkewbFLE4O2vu+WiAjDO21pM
qMI9SiFulbMC6hPTC2V52iASDpt1+Nbr9h5vi7yQzc41Fp5WUsZ+1WZyDpGuvC2c
GUGB5FfFYp1pb349AE7nt0GyWw9BASl7HATIgFfihSUpuUB1oK7mPxljTvMxDX+G
fT/QVfr7E6qcP+x73xXffcXEXPhgqBQIAOespTs+18L6do75tojjv9ZJhT1IIF/E
I3+XWokvKYEz2p+DqHBgy6WoobjBH+JWoDr09PT09PT09PT09PT09PT07/KiQx6E
nUYQehAUcNwB1DwODauPsnPobGVRCI5kQbFjDid/h+fOaX5ToQaHJxJskr9i77pu
jdLe1mTDGAyKUE0DZb4AeJeJJ8dyzn82YtE1Lr21KxuKiDP/UJBwlL7x49pRUf+o
BRCUaSDvWJZwpn/cZzXTENQabKPDQ7qWdTYhVoQEHt3laRiBFycOxKK4lPPJicJ0
oH2gPZnpE2fNh2duj/dr+hBLMifemOnxSjhYLF/cJlP01zNTha9bC2955F8r1pOq
dqEmQ23S6aJFtKEI6BgPen3D0jiI9ZE47v6920JKdh+jqF0SKIpRCz0C0Tkwet3A
4jeFcoX4HeNoxn1eWJnJg179vibR9CkK2mkUJwHhgaor0L15OXoA3BrYF3Tr8sgB
jxNj6013v96rmmHOpcJYNLwBSwlxJIY+tZ/JclwygChIAPflh4yZndiVYIUB9WP0
cJkF33Bkbyv3nsqpiZRewgFTUvT7GneOg87B3ZAv9wyY9h/KzkOSvFvGi5xB1eeQ
eI3nbxrXJfTgEgmeewP8j+/arlyxYlmu6inRQSjL63TDEU9eCi97JRzFATkYAgD0
Q4L6WV534Qal1Ud/lAUURoftJp6Uuc4fBV4i2AuAp6rPMOXucLD2OMK6i/vG2MeX
TtS8iVlYcS+AW61uiY6udk8Uj8Tu2zuKy0XhFRQRJAiIFvfDgS99TYWWvqZfWreU
nNVJfAL5oEr/IHu/nsWuwb+Vxger+fplC0LPktued3T8WmSBcrdk/AzKgxv8Dbv1
O7ANEckhlKQKpxh21sE7fukH82UAf/5Abp5Cj+zp0Ct+fcsdXkw8N0ZaV6SiK30F
vWifW53JY/UHrCpQMU4XC7RpQSI8aO5mfeO0q9G6fYh6t6n4NvHt+aC8JKqH5Kwy
KQbhi5gHd94yPrYTF0EcPVMiw+yOvFVBlA254x7V+XSWn0zdE8DL10TThvI0PGWk
3iogyEt33e4X2fpYW2RQD0Y+MKf0v6UylVbRHt/5VM2kb4hpaE3me5WafudvrCaD
z1ud79rtQQLrNPTUidjU/XltA6XSvFx04qf9COrfvE4PgLMJPZIOH9EIEq/8/Gwr
8ZoRapLQlc5RxeXYUtPf0FhO/TdrDb08fVIb0mMMDIKq20EpEW85gQOFShTUxZcY
WUVYdUBVcG6uX7yCVj5N4DipsQaU3tDCiYMzLx/sivf1EisvteVHi9GG9PhFRa2g
BgirldWWSTBw7kpomAeaW+2XZLKfKXQs9DdCTio1r7aiML8zh25b4027+a40RT/F
BkOHZ9gXP+b9Dd6XBIKv3/17MmZ574uyoVgGzNoLm+dLgnXiVgExBepyMMcNjnIT
xpF6nvbGDLSE0UIwkUPUUGoEZMGEVNkP1WTuUyd+KPv5wYAU/gq4jopy4+GjilKo
T5KihA7TE3D7bzsF4wa6HO9yhDTjK7sEw5H/grpQxVOKSX9PBI1o+LS+20xm3NXq
Nq/4Zbst3ZUtmYdW089xn0Jyrrx/j/mv4n2G1e/bf+e41S6ZW/gpdLkNveNF3QOg
Uwgu4kW7JVjlBsOeDh50mDzIyvCpWT2D3cqT2DEhB3hz7d9OzW7j/wHF+D0XLD/H
JndVqrhcPW1cX5zHm63z68A5N+kUAP1qF2pbU7Sx2wPXNPUDRYscuLS+NdV8vcRA
mKDp0kFN4gcdyCW1uYSAFvc7HvpOG5IMExP1fHX/1pd8SFLX5S8V+l+CCiNTgfHf
B49DnzMXGAYkP+UPBzNhqTNMpWuCkzPWN/pk9/KmQE1uL3808VRTaw7OGj3XqLND
GadDJEW5dI8G6rsDRCICpQGpD79DkECjH2oP+7BLcYd0vvGvp7s4Z/2pXlxrvUA3
51fpC4JN1QGkRsTvZphcItYLmQqqDy1+TKE+vouobZb4K1tibjH+4A+vgIZyPEs6
2q+hsWcfau94WDoK5wvJSuyrYAInz/FXKG7ZgaeeD5Jh3w+K7xHrBqHSGt4WrOVO
pZPGajC52XsmbMnjCIxGxiqyavE8PlRajqoQnOcLzw51DorXDDXmhtpEpFyRJFmY
9BttiCsTcyG51CpMLeU4igG4kbR/5qORp+J5lmAOkdB/dz/kuofS6MLdP7Pilu+W
p4ThBUn2QadkiEBU4mfSKelAxBPjSI966olOmTnhynGGFc7Zgs013oqVRzJ3aBXy
zhX56qQ5Wf/Xlove6Lywnhpxe/pI/hdIXxcW0WgjCwEX1/umvVBKoTuuIhzKWu6/
klST/d0DbLilW09Sy6yP+OwuhL64Z24NLinZEzNHG+H13BiJJApmnQvUlQ8u9mQ2
QCCAoOMnGWX35SuUzBRSHO54Af7xGiR1HJFYzfvDOn1asIB+fiJhKBri40hrECHk
/+aooDacNiKUT5vC8B5AMYSu/vRc+LDywuMx/yvGey1NhFWh+x5g5+kjc6MPwahq
GbadCfcz66col/Hi/xryX/QACnmP4vkvzf1LFN/jXmM9PxzrwiUiUceprdZVD6vv
XrzcaMWbznEJdx3R93Kejcu5vGYmuRJ8pxf/1Pd+gXwSIQdYEXymRG62C4g0hpV3
Odw1otzSV/FIeXBderZP8EOH1IBpqBEtQ1yZnbEAL3wtQ9p1XX5agQVGRUL7LzYk
tL2QiuqYXCfhgVbpgU2RqxKE2fEPOYlocVVjmiuKwTkagkNqnZZ+dQSctjL4eSbR
fYynqdAKy/x8aMjKw0+IEaJhenOUhiDNHDpIADqNcdWv3z/eKtTzLsEXV8o9gSir
S9I34Ze3EG3nYp9yypPOk+rX7KFcxHyNdLviqBxP+PlScMbMWXIBTjhj2MkEZyZf
XK4XFI+q1C6sFzwMoLteU3i183fKp2GcFGLQ99usaBn2M+uuUrHs0/+HXcukryhv
jpIQTcB+ApB2zkw5/HviH1Lg1/g104+ZzHfhv04xd7nlrFJwqM1n7ILHI41hhYvC
fp5v7Vfmz+lGDtz/sbx/UgkETboc/qTDcNDhMQDaIv83jSXT/39Y+6BUfvYoP/nU
9Rzf/cpIBuMt0OPqaKHyu34A1k73cUhI80bRJ4IFn4zed+VmiKsFzXLq058waTdI
2j4sFXUZRd37AMSF63cCvtCYP47vFs4tmj9dN8dxTwz6H6AIx2YnPzwoilc5/5zo
dMqXeHhuAlu8UCA9Lm1+Kf/OijgUy9GayWKVKPk1QC9S9pF5wyZ53APCAh2GLkWE
kHKksXJGHJuBJD8/ENJSIUct/Mb+u0+HxHsXP48uF+JNl5gIswY/a1mS4JGZU4Nf
N0KknVpvEtEXksr2IjFRQ+YnILac4ZWZXE3DfbykHrh3OGWQZfB2rafaWG7GydRY
neS/Xp51RRPkG4KT9IDrTq4nTf6mAoWB7n0o9gSiayV7gF1mcl9d9t8umvXNUG8Z
/Ur8RVypzpSazdeBfUCUCYh5Bon85nPbqnUIZv48rMCabh4kzbNFICAReG57in30
th2v/0b1fUVGNRvI5Orhov/z8AdshreGNYK8FUaLMUB1BoR/xqKlmq4n1qdLzrIm
lNpVdgRROS0/yg73Saaw6FvCf5Vbeo3/MDXJvUbYe+6+huTap9pAilC9A2O0EMbS
OctJ8uvXGRXBc/rne8JMuO03F1tLdOua6WxHhkDAgQK1wCL8PmWRRPVr4qOViXDd
tg+GSHP7SwUJQ/nEylO1yUXTuvipaF7XzpfJL19k42XHStuPyADeNqQSe3d1N0uM
bgmtZxCHl1pEuq6bztcCQFXSc2K6haHJUfVfxa886ZSnKv6H9jKqtXA01edq0OJt
M1knwWneegaSwOh0BvlByFI/f05yH9E7bTLJEm1MNTiTvgHF7B1v7lym99Gm6ZCm
/9vFJkNUodbpeqblv/49ga2FjAsSyxRVLnNOf6ui7ENzmu1n6TEoP/+ANp1qa/n5
S34u0GjYuCu29yuBR/V9wQETkQ04owOmDKEbab9GXmyqZigpzsRq2/f5GZuTxa7k
eZnk+GYVfpk1Cqt/3XiFrYZ7EarUf8YI0aBZTwl0tIBIKJBTifaSRfBWG6tFXZGj
Z0Y6SoYc3YB3rB69Jnr6C0G4J1SlitqS4vGzx/TuOSSnpEhqedKKYucPwuCAxF9A
b9iEIkeoK9S7Xr6zmcyINvgVb2H8PKO8uQ3PlWyqG2KuJBP7hUhad/ovae7WcqHV
P/Glbczf8n3/1D0EkKZC4yYjcz6OntxBuKSTLz0rB4bChqr3zOLKrzce/qVfibC6
uWZNGRu7psPXyZ2UD+ZxZzvSh+pDHDFneSYo9Zahvtgf+9XxMVKfn0p8tZ5p3YdR
uPkFCfs1KclX13pO734scfH6nn1UP1lYEGynL+QRYSZtIPFJeuv40K7PtGyDoQcL
kq+OhJmnoepmLUAeabQnmvwNfIV+lteB9bM2MddIneQ1WJd8RwZkNXjCdi5eABtm
HYb6Xr7cdt5+mJVkC2n8QjlbKGxF54f/eG2tHL0HvrUrb4q7RGqZw64mczKuR/08
Y6LmLRqt7NGwXe4muwD7Jus4Q3/GRgO59sl1KIyYuFOVoD8VIfOpVlNC3OKENddM
0L43b1xwirzKMvkedLtKZ3Cm21///omP9gLrLm58ShWoPg3S+aYZxX8yBWBNzVWh
GZleTzfLRaLHR+MiWiXKjS74dbYFu5x1xHCla5x+/ihL8TnJVPGVoq1DIY/D6/Nk
1F9ITwJVGafH2ynoV7Uu3fN7d27EAwIbWW/z+eqGaWyUyHP4/KN7L4PBuBfjhCiw
GnyJogKo9nkvTLCtMD9JscFYkzeKrpuz0Zs/WHWHbsmtFanP40+Z7IignqRKcbUX
SWm3NuP6FhgGcm/64wpSo62LelRyuzt5FJvIacKjdUzlPfJK1XCrKlD7J/AO1ETC
zOrh2KAs+v9wBa3q3zQBL6eyHOw5zc4d9BfhddtfSNacYD27/S+ZhhKlXgllsagI
dczgn56PKU+jg7Nhv23JnMU1kpuvOO7I/TMJCQbB9rdcnIUNBzg/paB9qjkZXTLJ
JYHd3QRyk10UF3fA8yMe6IrdGt46x3So8nDPtea4ocweIpduXxVeDumwHiYGtRQW
CcbEkgByARqlzmt7u+PsNqW/T9Kk1U4Wy5yf8X5LJ6PY2p5n+ZEn5cT9TrqCc+tu
era4+PHlYufEdRu3Rl35Yf6m4jv54cygFf1fmirm0dHoxKtsvLzPgbEyI8piu1UQ
LJxcHqsukPpg1OOi5dsi90EPZgMyfRu42GpZkQ2u7dLXLKf+eC2a/idghjncJfa1
3Jivp4GgBMjR0vib+vgTQAAZZaIH54iwaGuzLbBsoNDPjL6QrRyayhFynV+w94dn
BhVQgLc+6yjwyP6m9DFNr7EPEouACxk9Ri5ETIzfL67gp+vh1/S3u2MX7qpeCX/E
+jvKVEwqh0r3G/0go6qNlQJIYznTPHpfHR/pApv4zFJ6EE+g6MYKoSp0R66Fvxqg
AAD/mOPJ5y5pL9OY8Ro7BmBIbAfNF/XM1ynqtnMroPLERjnAAAAAAAAAAABYTVAg
WAMAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9
IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8v
d3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJk
ZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6ZXhp
Zj0iaHR0cDovL25zLmFkb2JlLmNvbS9leGlmLzEuMC8iCiAgICAgICAgICAgIHht
bG5zOnRpZmY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vdGlmZi8xLjAvIgogICAgICAg
ICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgog
ICAgICAgICA8ZXhpZjpQaXhlbFlEaW1lbnNpb24+NTU3PC9leGlmOlBpeGVsWURp
bWVuc2lvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjY0OTwvZXhp
ZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0
PjI8L3RpZmY6UmVzb2x1dGlvblVuaXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAgICAgPHRp
ZmY6WVJlc29sdXRpb24+NzIwMDAwLzEwMDAwPC90aWZmOllSZXNvbHV0aW9uPgog
ICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTExLTA5VDE0OjAzOjQ1KzEw
OjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8eG1wOkNyZWF0b3JUb29s
PlBpeGVsbWF0b3IgUHJvIDMuMC4xPC94bXA6Q3JlYXRvclRvb2w+CiAgICAgIDwv
cmRmOkRlc2NyaXB0aW9uPgogICA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgo=
" />

As you can see in this diagram, an entire shape of values can be represented with only a single value and the others can be broadcast back out when needed.

This is actually a simpler implementation of a more general and powerful concept called `Compressed Sparse Fibers`. The amazing library `Apache Arrow` contains performant implementations of `CSP`s.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://www.boristhebrave.com/2021/01/01/compressed-sparse-fibers-explained/">Compressed Sparse Fibers</a></blockquote>

We can ask for the equivalent `numpy` array at any time.

In [19]:
# ⚔️ Runnable Code
private_tensor.child.min_vals.to_numpy()

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

As you can imagine, a single `scalar` and a `shape` tuple is going to be much smaller than duplicating the same value up-to hundreds of `millions` of times.

The important thing to remember is that we are retaining the `shape` of the underlying data.

In [20]:
# ⚔️ Runnable Code
assert private_tensor.child.min_vals.shape == private_tensor.child.shape

#### Data Subjects

Here we can see a representation of the `Data Subjects` for this tensor. You will notice that the `Data Subjects` array also has the same shape as our original underlying `numpy` data.

In [21]:
# ⚔️ Runnable Code
print(private_tensor.child.data_subjects)

print(private_tensor.child.data_subjects.shape)
print(private_tensor.child.child.shape)
assert private_tensor.child.data_subjects.shape == private_tensor.child.child.shape

[[DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'}]
 [DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'}]
 [DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'}]
 [DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'}]
 [DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'} DataSubjectArray: {'Ishan'}
  DataSubjectArray: {'Ishan'}]]
(5, 5)
(5, 5)


<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### GammaTensor

If you were to perform an operation on two `PhiTensors` where any of the resultant elements was derived from two different `Data Subjects` you would get a `GammaTensor`.

There are two primary differences between `PhiTensor` and `GammaTensor` and they relate to keeping track of different `Data Subjects`.

<img width="500" src="data:image/webp;base64,
...
" />

Let's get a second `PhiTensor` with the same shape and a different `Data Subject`.

In [22]:
# ⚔️ Runnable Code
second_tensor = sy.Tensor(data).private(min_val=0, max_val=10, data_subjects="Ivy")
second_tensor

Tensor(child=PhiTensor(child=[[2 2 3 1 3]
 [6 2 5 1 6]
 [4 2 3 5 1]
 [4 3 1 4 4]
 [2 3 1 3 5]], min_vals=<lazyrepeatarray data: [0] -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: [10] -> shape: (5, 5)>))

#### DataSubjectArray

Okay Ivy, let's see what happens when we combine your data with Ishan's.

In [23]:
# ⚔️ Runnable Code
gamma_tensor = private_tensor + second_tensor
print(gamma_tensor.child.data_subjects)
print(gamma_tensor.child.shape)
assert gamma_tensor.child.shape == (private_tensor.child.child + second_tensor.child.child).shape

[[DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'}]
 [DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'}]
 [DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'}]
 [DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'}]
 [DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'} DataSubjectArray: {'Ivy', 'Ishan'}
  DataSubjectArray: {'Ivy', 'Ishan'}]]
(5, 5)


If you look closely you will see that for all the element wise addition the new `data_subject` values at each index are a `set` containing both `Ishan` and `Ivy`.

Let's take a look at what `GammaTensor` does to store the `Data Subjects`.

In [24]:
# ⚔️ Runnable Code
print(type(gamma_tensor.child.data_subjects))

<class 'numpy.ndarray'>


Okay, it's just a numpy array? But a `numpy` array of what?

In [25]:
# ⚔️ Runnable Code
print(type(gamma_tensor.child.data_subjects[0][0]))

<class 'syft.core.adp.data_subject_list.DataSubjectArray'>


What is a `DataSubjectArray`?

<img width="500" src="data:image/webp;base64,
...
" />

The idea behind `DataSubjectArray` is provide another level of abstraction which allows `1` or more `Data Subjects` to be tracked for a given `scalar` data point and for that `type` to know how to perform any required `operation` to itself which was also applied to it's sibling `child` Tensor, and optionally another `DataSubjectArray` in the case of `infix` operations.

**Note**: `PhiTensor` uses the same data type but the `DataSubjectArray`'s benefits only start to shine once we create a `GammaTensor`.

### Operations

We can also prove to ourselves with an `assert` that the underlying `child` data that was `+` added in the `GammaTensor` is the same as if we had just added the `numpy` arrays in each side together.

#### Add Child Data

In [26]:
# ⚔️ Runnable Code
assert (gamma_tensor.child.child == private_tensor.child.child + second_tensor.child.child).all()

In addition our Tensor chain has changed a little bit:

In [27]:
# ⚔️ Runnable Code
print(type(gamma_tensor))
print(type(gamma_tensor.child))
print(type(gamma_tensor.child.child))

<class 'syft.core.tensor.tensor.Tensor'>
<class 'syft.core.tensor.autodp.gamma_tensor.GammaTensor'>
<class 'numpy.ndarray'>


### Add Bounds

What about the `bounds`? Do they change?

We added the data together with `+`, so does that mean we add the `bounds` with `+`.  
In this case yes.

The `lower_bound` for both `PhiTensor`'s was `0` so that doesn't change much.

In [28]:
# ⚔️ Runnable Code
new_lower = 0 + 0
assert (gamma_tensor.child.min_vals.to_numpy() == new_lower).all()
print(gamma_tensor.child.min_vals)

<lazyrepeatarray data: [0] -> shape: (5, 5)>


The `upper_bound` for each is `10` + `10` which equals `20`.

In [29]:
# ⚔️ Runnable Code
new_upper = 10 + 10
assert (gamma_tensor.child.max_vals.to_numpy() == new_upper).all()
print(gamma_tensor.child.max_vals)

<lazyrepeatarray data: [20] -> shape: (5, 5)>


If you think about this for a moment, it makes sense that what ever the highest and lowest possible values were for either `tensor`, they are now the `sum` of those possible values for the `sum` of those `tensors`.

### Add Data Subjects

And likewise, the addition of `Data Subjects` works in a somewhat similar fashion.

In this case it turns out that because the `DataSubjectArray` for each `scalar` is itself a `set`, the operation that we apply is `union`.

In [30]:
# ⚔️ Runnable Code
first_private_tensor_ds = private_tensor.child.data_subjects[0][0].data_subjects
print(first_private_tensor_ds, type(first_private_tensor_ds))

first_second_tensor_ds = second_tensor.child.data_subjects[0][0].data_subjects
print(first_second_tensor_ds, type(first_second_tensor_ds))
combined = first_private_tensor_ds.union(first_second_tensor_ds)
print(combined, type(combined))

assert gamma_tensor.child.data_subjects[0][0].data_subjects == combined

{'Ishan'} <class 'set'>
{'Ivy'} <class 'set'>
{'Ivy', 'Ishan'} <class 'set'>


If you imagine for a moment that we did a `-` subtraction instead of `+` addition, who would say are the `Data Subjects` who contributed to that `subtraction` result?

We can't `subtract` them (have the `intersection`) because that would mean no-one contributed to the result which is not true.

Think of combining Data Subject's more as `mixing` two 🧪 liquids. They both always contribute to the final liquid no matter how many times you keep mixing any two liquids.

<img src="data:image/webp;base64,
...
" />

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### Provenance

The second thing that is different in `GammaTensor` other than the expanded `DataSubjectArray`, is that we need to keep track of every subsequent operation that occurs with a GammaTensor in a dictionary called `source`.

You can think of this as a tree which keeps track of which input tensors combined to give which output tensors. Just like with the `mixing` example above, we need to know what ratio of different liquids were used at each stage.

The key in this dictionary is an integer (unique to every GammaTensor) which maps to the corresponding Tensor which was used as input.

We can see below how `gamma_tensor`, which was created by adding `private_tensor` and `second_tensor`, has both of those tensors in its `source` tree:

In [32]:
# ⚔️ Runnable Code
gamma_tensor.child.sources.keys()

dict_keys(['1058584437', '461235248'])

In [33]:
# ⚔️ Runnable Code
gamma_tensor.child.sources

{'1058584437': GammaTensor(child=array([[2, 2, 3, 1, 3],
        [6, 2, 5, 1, 6],
        [4, 2, 3, 5, 1],
        [4, 3, 1, 4, 4],
        [2, 3, 1, 3, 5]]), data_subjects=array([[DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}],
        [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}],
        [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}],
        [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}],
        [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
         DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
  

A `GammaTensor` also tracks which operation was used to create it in an attribute `func_str`.

In [34]:
# ⚔️ Runnable Code
gamma_tensor.child.func_str

'add'

As you can imagine, as we do more and more operations, this source tree can grow quite large.

This is why it is important to think about and seperate DP tensors into either a `PhiTensor` or `GammaTensor` because if we do not need to figure out a shared history of operations between more than a single `Data Subject` per data point, we can save on compute and memory by not tracking every single operation and a copy its input data at each step.

Many data science workflows can and are being done with data belonging to just a single individual, keeping all the data needed in a `PhiTensor` representation.

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### Anatomy of an Op

Let's go through how we implement a simple op like `add`.

#### Let's do PhiTensor Phirst 😭

First, we need to implement the `__add__` infix method on `PhiTensor`.

```python
# packages/syft/src/syft/core/tensor/autodp/phi_tensor.py

def __add__(self, other: SupportedChainType) -> Union[PhiTensor, GammaTensor]:
        # if the tensor being added is also private
        if isinstance(other, PhiTensor):
            return self.gamma + other.gamma

        # if the tensor being added is a public tensor / int / float / etc.
        elif is_acceptable_simple_type(other):
            return PhiTensor(
                child=self.child + other,
                min_vals=self.min_vals + other,
                max_vals=self.max_vals + other,
                data_subjects=self.data_subjects,
            )

        elif isinstance(other, GammaTensor):
            return self.gamma + other
        else:
            print("Type is unsupported:" + str(type(other)))
            raise NotImplementedError
```

You can see here we essentially have two tasks when implementing an `infix` operation.

The first thing is to determine what type of input we are using as `other` and then based on that, we need to `normalize` the types so that we can use them together.

If the `other` is a `PhiTensor` we turn both the `self` and the `other` into `GammaTensor` and 
then ask it to handle the `__add__`. If the other is already a `GammaTensor` we can just convert `self` and do the same thing.

If however, the `other` is not a `syft` `DP` Tensor we can do a kind of best guess `broadcast` type action where we add the data to `child`, `min_vals` (or `lower_bound`) and `max_vals` (or `upper_bound`).

#### Now GammaTensor

As before on `PhiTensor` we need to implement the `__add__` method on `GammaTensor`.

Like before the first thing we should consider is what do we do if the `other` is a `PhiTensor`. In this case we just convert it to `GammaTensor`.

If the `other` is not a DP Tensor we do the same as `PhiTensor` and augment the data, but since the incoming `type` has no `Data Subjects` it won't change.

Finally, we now have a new set of the four main attributes required for `GammaTensor` under the `+` operation, `child`, `min_vals` (or `lower_bound`), `max_vals` (or `upper_bound`) and `data_subjects` so we can `__init__` a new `GammaTensor` and make sure that the `func_str` we just executed, `add` and the source `dict` that went into the `add` is passed in.

```python
# packages/syft/src/syft/core/tensor/autodp/gamma_tensor.py

def __add__(self, other: Any) -> GammaTensor:
    output_state = dict()
    # Add this tensor to the chain
    output_state[self.id] = self

    if isinstance(other, PhiTensor):
        other = other.gamma

    if isinstance(other, GammaTensor):
        output_state[other.id] = other

        child = self.child + other.child
        min_val = self.min_vals + other.min_vals
        max_val = self.max_vals + other.max_vals
        output_ds = self.data_subjects + other.data_subjects

    else:
        output_state[np.random.randint(low=0, high=2**31 - 1)] = other

        child = self.child + other
        min_val = self.min_vals + other
        max_val = self.max_vals + other
        output_ds = self.data_subjects

    return GammaTensor(
        child=child,
        data_subjects=output_ds,
        min_vals=min_val,
        max_vals=max_val,
        func_str=GAMMA_TENSOR_OP.ADD.value,
        sources=output_state,
    )
```

#### DataSubjectArray

We also need to make sure that the operation we want to `apply` is implemented for the `DataSubjectArray`.

```python
# packages/syft/src/syft/core/adp/data_subject_list.py

def __add__(self, other: Union[DataSubjectArray, Any]) -> DataSubjectArray:
    if isinstance(other, DataSubjectArray):
        return DataSubjectArray(self.data_subjects.union(other.data_subjects))
    else:
        return DataSubjectArray(self.data_subjects)
```

#### LazyRepeatArray

Since our `bounds` are usually stored as a `LazyRepeatArray` we need to make sure it can also apply the desired `op`.

```python
# packages/syft/src/syft/core/tensor/lazy_repeat_array.py

def __add__(self, other: Any) -> lazyrepeatarray:
    if is_acceptable_simple_type(other):
        return self.__class__(data=self.data + other, shape=self.shape)

    if not is_broadcastable(self.shape, other.shape):
        raise Exception(
            f"Cannot broadcast arrays with shapes: {self.shape} & {other.shape}"
        )

    if self.data.shape == other.data.shape:
        return self.__class__(data=self.data + other.data, shape=self.shape)
    else:
        return self.__class__(data=self.data + other.data, shape=self.shape)
```

And, that's it.

As long as we know what to do in the following scenarios we can implement any given `op`.

- What happens between the `op` and the `self.child` data, and optionally if it is an `infix operation` the `other.child` numpy array? This is usually the same as the `op` would have been in normal `numpy`.

- What happens to the `min_vals` (or `lower_bound`) and `min_vals` (or `upper_bound`) under this operation? Do they need to change? If there isn't an `other` does the `child` data get applied instead? Is the `op` defined for `LazyRepeatArray`?

- If there is an `other`, does it have `Data Subjects`? Is the `op` between `DataSubjectArray` defined?

- Finally, what other `args` or `kwargs` like `order` or `axis` does the operation support and if so, how are they reflected in the above operations?

## Publishing in Syft

The way a `Data Scientist` interacts with the DP system in `syft` is by calling `publish` on their results. What they are essentially saying is, I am done with my computation and I want to see the results without having to go through a lengthy process of approval. By providing a desired amount of noise expressed as `sigma` or standard deviations from the underlying distribution, the system can calculate the amount of `epsilon` required to protect individuals at that level of noise. If the `Data Scientist` has that amount of epsilon `budget` available to them and they have not previously exceeded this amount for the any given `Data Subject` under study in the results, the system can account for this `publish` action, deduct the budget and release a noisy result to the `Data Scientist` immediately.

### Publish Walkthrough

Let's walk through how publishing works by doing it step by step.

<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:10px;" src="data:image/webp;base64, UklGRt4jAABXRUJQVlA4WAoAAAA0AAAASQAAYwAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBIkQUAAAHwtm2bYdu2rQ/peiIyS6mtdNu2MWzbtm3b
tm1027Zt2zZrKREZcX/ILKWVklkiYgKY2JVF4bwDM3M4Gj11o9FRN2ZvpfeFo+7M
HG/94+//tiUeMJb7xk9++jnD1ZzNDu+9BzDn6PNNKetQnEHBV5SkQ8FoqfOOBrOC
dR9TrnQpGJT8VkFJb6PXFvOMcT1+rZCjTscafqVRjrpqAWvJeLOSdR9VVtQRuIbv
KkhBX6Rslxklv1aQog7FU/t8LemWEmsXBes+qiRVutxjtd/VFPUVijY5o8evFSQl
PbAKDgoOUpRU6VqwNuFZ91HlhrtXwBmewxpU6UOU7TGj5DcKklTpSofV9lOoJd26
Im6MzQw86z6q1HQeZlCwa5OifkjZYEYLe/xGQU3nNpT8fUzSw2vgWuNZ71GlMWeO
+dsYBf2cstbKkj8oqjHqRAwo+Me4pMc3xLfEsdGi8rjj6fd65YC/jFPU3ylbUvJn
BY07nOZvajQm6cEluFZ4NhwqjUm66s2ve917Xvf6Vxysaoyivo9vRcn/FDVhjsNh
HA5HSRNm6bn4FpRsJeVJphv0Y3rTM2tyLJysShPnsRMlXbkzblo2zvNbRc0+6A8U
U7IJHMe1Imq/aTmDJmO565RakJWeQTEdB5gDPOuNWqGo4zBrKnq9nm8wzBrM6PET
RbUxJ72JosGouwYz6s7wrP6wcitUaR98zbH5e9/3zlXxk5gZBX9WVDtzfnhnCih4
24MhhpufhYHRbHg2e1ypJap0EpjBhUqqdEgNMAMoWPUqJbW20vvoO7ZXTjnpkTWp
mwGY43eKam/U6dDnawqSgt5VM6PueOlQuUU56ZM4zlQlKWp/wAwDPDsPldTmnPRB
tg7KkpLu7VM3B0v4noLaHXQ4O2vMY4MGHJ7V71JqWZZ2WFW56ZF+E54dL1BS25Ou
/milWlZ6UoM5uFpJ7c96JKmeNdqggR5fUZU7oKzmrLhcQ48n5pzVyTyuWr7mWet8
Ver0JAcqqmtxOaDH25Vy5xaXA2PhclXqeBW1PJR8W0EdD9LtPRyD21R1LEv/3QJK
Pq6obmcd8wIPRv8aVR1TPGMTPAXvU1TXg35JScE+cyDpegds/JBS53LS8zDep6Du
B/0Zx78U50DS5SXLD7U4Sh3LYVFhdVY6QZJyp7IkXdSHFZ786UPuVe5Q1uMX/+Ut
G4DBpp86t1NJl35oLTBg1/sfT+p29cD1ew6AjSrNxWdjPENVzl3LQe/G8QpV6n7U
Z3G8VXEOBH0dxxvnxJdwvGROfBjHztKo6lg1ino5joWLJOVOZUn3roF5VnzKZ89W
7lDWuV986UbOAwsv+v1VXfv6joBt+NfLHpOyOn7fWb9cn89oXv6OF6pKOXcs5xz0
GVZ9QElzMOlJcKTiHKh0bQHvV5gDQb/Bser9St3LeiLm2UOhc1HnYXier9S5oI9T
gLGfYscqXTrAoOD5qnK3RvoYJWDGUQqdqnTtAAPwPEHKXYp6MwWNnl9q1KGRDsRo
NgbXK3Sm0nBD/Bg8z5aScgdyzpXei2dCz0cVktqflRf1UxyTmvFPSbl1StLxDpsI
R/+g716i1LJKp/zw0jVwLKWxAocrtizqcNbBWHrHnh3YF4wp9thDiy0bandKpul5
tTRqVZDeRjEVjM9Jw9yeofTLJdh0gFfdpxxaEis98gamb7DtntIwtqAaSmc+mVka
9N55qxTjjGKU7vnQ8mAzwAw2/8tdUhhV08pxFKWw22ZgxmwNbJNv3CqpCjEtTU5x
lCVd/6WdSzBmbgZs9O0jH5GkahRGoYoxVmE0GiVJuueQr2wAmNFGM2DJNp8+7EYt
/fVHfXjjJYAZbTUDGKz7pC//+z+HXPDQbbfffdtFpx/2/48/a90BgBldNLP+Ggv9
wRp974xZAwBWUDggdg4AALA1AJ0BKkoAZAA+LRCGQqGhDUabVAwBYlnAMkIT/+Nx
x/L0j7b/zAecT6KN499ADpN/3R9Ju7a/m34vftf6o+Any3+qftD/bf/D7eX9n3ru
YvMr+SfaT8n+an9q9i+8/3ef1PqBfjv8j/uP5g/mh7VeyZ0j+++gF63fS/9f/eP3
j8y7+69APrJ7AH8z/m3+d9Q/9r4GH3H1AP5Z/bP+B/kv3F/xv0pfzX/m/1H5b+zL
9D/wH/X/yvwC/y7+m/7D+//vJ/lv//9ansK/bX2T/16dagQq2tkungmcZ4pajjuO
s9LdslFQfk+y2sLhqtRkxcOkDFPPMIqB8Y/zfUbkquuifdS/t/GuCPFuyVsJcoCW
PfltIxzcmfWkh17yI1Ucw8VZB7fGgXxT7I1KF19g0M3APOMKjCmHlkGCuvfg3sUC
3CoM3AMvjUy7Rv193uOGKUL2ySECgi6E0OwjvWd4bT4Rw/HDZwPH86iVXUDFJwl/
6lxy7Mw3h5yMq0lVyBg5IPdsxfuFYfegMEPs4Fd6N5QOIBx4MiqWaVEBCLyT9Vxw
CR4LkMS5BnhK6/sr2IKjhf8WoPfIAAD+//7kQ90SX+2y+Kd1a1Oic564eoBRpV5O
pkZL5CHReAz47mEyXiD/yMCC3t9r6x1PoNG0BJ3qUCz90hvAtv3nF0vhDrnZI/6G
70Gb9MoXjDxLmyOGE2nS6G0tdSvO4yZ+4OQGig6HqLQ2q7Ec2aMhkkjG+abiL23n
28esodwafKsdkUMBJkjh7ek5FEh9za22+VQ+3XOsYvP8WgJ+8mcaWHN9jhtI3Ep+
cm2Ohy8D6ctEUm0cPj7hA77jIP/045gl/vWTSqfMiZNatJT5mrKf4/IHb4RqFt1J
fDeTLfnALEH3Ih7Bc6OIwHIpln+s5pmhCAtrSgG2Gi4x7q/8ZLuJ5veZd5LX/pNd
eQ4hDe6JA1/+uMWzrJwpu+sQqiHqN1hFklJ+9217YiaOfkvfhAp8NkpYySye2wiH
D3UAyfNyWTVgIw0adMjx0srgemGi968wV1fPP47VDOcBIE3P0O3xvxmlW6iKXW1B
01hqToPsjb6jvx5EJnqsR+3x8PGBRkz0uEt12h5/VgOv//hOMvltD8iciit3QWvl
DuoqOj4ZSsr6kQdXmahGCzgFmUC9nqXSwXRVjuBssRJfVSkp54QcH1n+QsJupqfm
ZndR/6HWZxUqA5RlXFi2Ly68HROFjtzFHFRIgbJ/8oKNIbHu4WWt3wECq4t8pCPP
nHlbPjv2ZfOnHI5T1+HvA3y1kPuRPmwMAz7Y2SywW1hX/lv4Fg6r0Unf0bpa8Zzy
8TSniXG1/8Dk+pcBI+YV+u+fxQ5g5P5mgWZ/GfTxOnKSCg5SasPPn1Waaa38Cr9Q
B6CgNg9/Et+FJw/acGGs7BPMJY3T/yUo1X4fL0wwvSLfT9fiYrjLjTrjOeBQerGV
MOpaRb14Dsz11Zx7jyEa5+LgKgmyZgV5AY5jgpLCXy2XnJp4ElxnOEQk+PdmXT9X
1Pm/+1mG27JCZ55yfgAxo5WNr0kC1ZQ3j3RIeH4f2xna8n1TfLB4bTFRu5irmHZL
xD+VGhoxRqxt/0J4TrVUxiZIQwXk7zbCWrhh1l7GuPcmDtODKscaJLrutRWjaUoz
HEWx5QVQHS3bR6uUMkae9CV0xmWBzwTg51VTo5KD6cI6IPa5kNqNSqVkIZ9iwbbw
IqsbhH5Pa/xQmEU7q4tjX6x6HOI4cBnc36FRLGq+VemQ0+Kmck+sgDNzOKgkeB/o
n4tLzpS/+15PdzIxCvBTU2WmW9qkTU2HAoCheQFgFONn+NggB+0l8y2mQayLQCfo
4nyTT8EopqwoYtpGLPi3NY4xZEO1lX6qJ/4bFrfeTSvI87awAFi+C8T70FNR6z+P
xDD/GcjW+QNaMD4jY2dwoyy/BRWt2F5tubcAaeXndOTFvLas9hmv8gGzxHPD6HuX
rGxIdvPFTUUfk6sMREWkfHh5XMtR/g8gU5+0U95/CqIW1rGqOxGRqZtwXff9CscU
e9abJ455Nm5IT2/lJvrcvLSK5LHhY4BjB6vayLl8ame6lLPYXyEWcooDCCINTzVY
TtycWAzKXb/ncUAlQg++6IjwXHwN2FufbFo+zlUu471hhkxHolrv9OJ4CLRD1dn1
dS7+1tZVl2f86IsILPJKCOxf+c6xjq0sZxpgIvv2hxOt0OoN5ISaoexqkNemE6VW
DtMoLjBvIjpFPNfsO5vLjkIo7JJRW8iqzYU0XY3kFGQ61N5KTJuRJ2y6nD90z82J
9LSjMMjZUwpbzXQgX/akwWEPUjBJdjCO9xhq6O0D0bIR6sUdYK7CSj73WNn2qrOe
ybU3rx0A2BVBGGK8a7iKsTE8FBO5J4Qc+P8P6A3pRS5a7zpuE1lu28nN+GpWsiCn
2OiC923My6FLy+KgWArbdqraJ1lahBNuayjR+EC+ZolY9ONU+4vt9XvQA0b9F+2b
hi/Iq2nxZuDAH83qtRdh7hksqCw3jrwaUBLUu2nwYAIcdI9ytkC8FSXA0lJCgxHY
q8cuvEBxkhTWUHCktF2xJWju65GbJ0xiJIQQSHC1oP9UhTt6hDM1vlGPz4aNri6H
2a1wekhi2srXkvEtWoGZtO8UUlCQ9zXdtXoEMBFnPXivAy4u2ldQjaRn7IpecSco
YuTg3ORtlAhdUJX7ZZy8n1cZmpwMEeQDN+iHbrorLj+JBZEpfkcDKF5W6lGEaKwI
mEl8vZTTviNu/kBBNCzWzd9EYYGQ8GyJkZjlOD/NeJzfyWvtaavhgO4oWM52YLYb
DN0DAzeQGTSGDN3hrw1/Fo6SZKxGt+gRAAt/m/Ha08BhQjc8k7nI//892lqmVTxm
3uliiiJFcvMOq+ULEBwvaUrnGUZyTodR+8+rATsoa6mvrr/+n5v6HCPOkZoHrIk+
K2G2twm0FOvM31nF+HvBIPyav5RHyjkpI+K1cotLlPMKQkR4thyZyz/v9jLDcSqp
m/IIVCTNq4L54deZIF70uU4ST6ODjpmLq96bBPOSDmCKKhhOmL91EymqNVQb//Dw
pUZnddl1EK3eMC/4mbQmjVkOweiArvmwBSoAAfh454Oa6JQ/2wJ3kpSilSlAIYnb
jJBubuiPko3CyeBiYXeUCBYks8CEoMar7v4ZaQ6whZums7QNm6WGqtYZ+ZY4vKQO
IMfgUtuxY9Q9tqdTH2kpJ157ePdVxgRDNkJf8kLSKulh4CK+NE/vYpgGJ/r40TJZ
0OIk/nfL1bsebhYFDm4DpQ+SYEmLxnkTrqswtS0ifpoCFGcncplU2cNYDsxpifLB
F+v+QwvS7/z/aiht2fgShGPB7FTB6AqD6in5LxpWmWz4ll5JZFYJpD88wDLH9A6m
KpfALqTiYWX2NFm/WmHe6Hi3ZCko8lrxrJ0g+wL0MPNKQyubEXa0HtdF2zaYRghu
B013JTGBt5RTrxf9p2ShE81hr9krhrYLbndkdQXwPtyF+86RVaiUzmbpy0QWye+N
gJIguTpX+kZo5TROK17S1J2GRE2AVy4vxJxlaQL5mefECzv4PZgx8kbYJ1g7zBC6
HB7IwcduUzPfKoj43kLTKsx/5a6oSH5DDJuy6B9NqkNrAaU5X8pRBOgn1+m9R5Ur
I0YRUnVqbUqrGX9Bl8w7/Ka3jljS8r88dt2Zk110kEoeAnwRp1O/3ft3PhrYWDpS
CRw01yVNT8aecFE1ZX4TH5mAiFCHT3uCUorspmL08V+Zw9pOdr0UnYb1VK8ifuwE
nUhLIsQIPJTo56qVeIHOn2SNN5vV2/+7oUz6T7jDLcJuD7dmK0t6g6SpFjB8/AX3
ftVxKiIL67YDShtNLqmaLpFVpRLkujIwh6Q3Shw7Dh1SPRBH2+urWgjVqJN6HGXV
OqBOmnHc7IXsT2br9bqSz5ZdPIYfGriJOtTcywEsgGKV9RrH/+l/aG8BBgGONoF5
ncC1+3uo/XMInqIT/p0uVccEYo/mAuAAzyfZkDEyJXbK+H3IJitFjK1FWFjEfAEw
lkXMA1BFO96jt1R3MkDQI58rZ5ozDwI7qip9iEdYVY++c3/GLkKeBwlwQlyK8fw6
GAB4AlhUcohKDJBZyylsRWsuhKiI3pvnv9iY/+AZD6Gox6fbXpP0pv/x2/9kp3/v
co6wGGUX2bQMCi5XYNrE4BnsBnKtUF7qKoZBuE4nHFxoRhEsOSQbXQ3/pjT+z6Dy
mtD89Q3IdCODPEebjf3GHPBlU4uvkSJUTR0WCmh2+J5u7Rg4DZUlymd+CddWb+eQ
VZq+YB/GsmyPLXu0AVvqWY6Qbq2Vb/sT85kP8SzB1xqZF0oIv9241PA6qyqt6QOz
pUVCaWzhwRo/mL3mhxUaD/2D3saRZRiNxBp5ywOToimXk9n9pDOQ3uFvPyVawfPb
GEiQ621mtLrUjPOPAwVSk1yWaYJSbvRgIepyq/k3+/4uh7kMyGVLc9Xw3+jAJg/e
YXQ2FmhEKOoPaffKQkFXpN+ZYx0YxHl//3N+SaeNhbjTkkU/LDNyz856KowbUB18
K7wqJqgeSCDS5ogotUMXQgshGeywI5JFbBFmZRoy56Sqo92SH/9feM5+iLugo4pF
c06LlV6v9yl15jbEaqH0DxoBK9RCDV5ZQOD3NsHaxL0hNLEclbiE/CpzO/SwvlUM
V1aVaicdNOllRL9D8zwQh343YmWwdTix+joUrh54G4+b4SOOOrpiII5Cyk0hnbJV
5XcK8TGl2hK77pmyhpy6sVw6LsG7X+yGyqcAOzbaMJ9N/z8G0rxIQ7CQf+Nuh/1m
BwhVWdRbctK10amxopDSwztAXDFs+icb4c6l4CFBDpnwgIy01rsKShqpC8AjhJH1
Rc8+N0BLkU2fD4A5mY3LPx/LE1CnlajIZ4IebwBDUrAmo7ogOhF6uHTFIa0rX0wM
R/+lzUmHuXp/65l6VAAKLJl8gAAAAFhNUCBXAwAAPHg6eG1wbWV0YSB4bWxuczp4
PSJhZG9iZTpuczptZXRhLyIgeDp4bXB0az0iWE1QIENvcmUgNi4wLjAiPgogICA8
cmRmOlJERiB4bWxuczpyZGY9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkvMDIvMjIt
cmRmLXN5bnRheC1ucyMiPgogICAgICA8cmRmOkRlc2NyaXB0aW9uIHJkZjphYm91
dD0iIgogICAgICAgICAgICB4bWxuczpleGlmPSJodHRwOi8vbnMuYWRvYmUuY29t
L2V4aWYvMS4wLyIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFk
b2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOnhtcD0iaHR0cDov
L25zLmFkb2JlLmNvbS94YXAvMS4wLyI+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDA8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8ZXhp
ZjpQaXhlbFhEaW1lbnNpb24+NzQ8L2V4aWY6UGl4ZWxYRGltZW5zaW9uPgogICAg
ICAgICA8dGlmZjpSZXNvbHV0aW9uVW5pdD4yPC90aWZmOlJlc29sdXRpb25Vbml0
PgogICAgICAgICA8dGlmZjpYUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6
WFJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOk9yaWVudGF0aW9uPjE8L3RpZmY6
T3JpZW50YXRpb24+CiAgICAgICAgIDx0aWZmOllSZXNvbHV0aW9uPjcyMDAwMC8x
MDAwMDwvdGlmZjpZUmVzb2x1dGlvbj4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yOVQxNjowNjowMSsxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgICAgPHhtcDpDcmVhdG9yVG9vbD5QaXhlbG1hdG9yIFBybyAyLjQuNzwv
eG1wOkNyZWF0b3JUb29sPgogICAgICA8L3JkZjpEZXNjcmlwdGlvbj4KICAgPC9y
ZGY6UkRGPgo8L3g6eG1wbWV0YT4KAA==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">System Check</h3><p>Discuss these requirements with your Mentor and if necessary follow the steps to complete setup.</p>
</td>
</tr>
</table>

Publishing requires a `domain`, some `private` data and a `ledger`, if we don't already have a `domain` to work with lets launch one.

In [35]:
!hagrid launch test_domain domain to docker:8081 --build-src=padawan_trial_dp --build


⠴ Checking for Docker Service     
✅ Docker service is running
✅ Git 2.37.3
✅ Docker 20.10.20
✅ Docker Compose 2.12.1


                                             `
                                         `.+yys/.`
                                       ``/NMMMNNs`
                                    `./shNMMMMMMNs``    `..`
                                  `-smNMMNNMMMMMMN/.``......`
                                `.yNMMMMNmmmmNNMMm/.`....`
                              `:sdNMMMMMMNNNNddddds-`.`` `--. `
                           `.+dNNNNMMMMMMMMMNNNNmddohmh//hddy/.```..`
                          `-hNMMMMMMMMMMMMNNdmNNMNNdNNd:sdyoo+/++:..`
                        ../mMMMMMMMMMMMMMMNNmmmmNMNmNNmdmd/hNNNd+:`
                        `:mMNNMMMMMMMMMMMMNMNNmmmNNNNNdNNd/NMMMMm::
                       `:mMNNNMMMMMMMMMMMMMMMNNNNdNMNNmmNd:smMMmh//
                     ``/mMMMMMMMMMMMMMMMMMMMMMMNmdmNNMMNNNy/osoo/-`
                    `-sNMMMMMMMMMMMMMMMMMMMMMMMMNNmmMMMMNh-....`
       

### Private Data

Let's create some private data we can use. This will be a list of 10 ages, and the names of the people these ages belong to will be the letters `A` -> `J`.

In [36]:
# ⚔️ Runnable Code
import numpy as np
ages = np.array([25, 35, 21, 19, 40, 55, 31, 18, 27, 33])
names = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]
assert len(ages) == len(names)

Now, as we have done previously we can create a `PhiTensor` by combining the `data` and the `data_subjects` and annotating the bounds based on our knowledge of the dataset.

In [37]:
# ⚔️ Runnable Code
phi_tensor = sy.Tensor(ages).private(min_val=0, max_val=122, data_subjects=names)
phi_tensor

Tensor(child=GammaTensor(child=array([25, 35, 21, 19, 40, 55, 31, 18, 27, 33]), data_subjects=array([DataSubjectArray: {'A'}, DataSubjectArray: {'B'},
       DataSubjectArray: {'C'}, DataSubjectArray: {'D'},
       DataSubjectArray: {'E'}, DataSubjectArray: {'F'},
       DataSubjectArray: {'G'}, DataSubjectArray: {'H'},
       DataSubjectArray: {'I'}, DataSubjectArray: {'J'}], dtype=object), min_vals=<lazyrepeatarray data: [0] -> shape: (10,)>, max_vals=<lazyrepeatarray data: [122] -> shape: (10,)>, is_linear=True, func_str='noop', id='353607843', sources={}))

### Upload Dataset

Let's log in as the `Data Owner` root user so we can upload our data.

In [38]:
# ⚔️ Runnable Code
import syft as sy
domain = sy.login(port=8081, email="info@openmined.org", password="changethis")


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into test_domain... done!


Now we can call `load_dataset` and add some extra metadata if we like.

In [39]:
# ⚔️ Runnable Code
domain.load_dataset(
    assets={"ages_tensor": phi_tensor},
    name="ages_dataset",
    description="Ages of a group of people"
)

Loading dataset... uploading...🚀                                                                                                                                             

Uploading `ages_tensor`: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 14.05it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


### Create Data Scientist Account

Okay now we can create a `Data Scientist` account so that we can play with `publish`.

In [40]:
# ⚔️ Runnable Code
starting_budget = 999999
data_scientist_details = {
    "name": "Rey Skywalker",
    "email": "rey@skywalker.net",
    "password": "jakku",
    "budget": starting_budget,
}
domain.users.create(**data_scientist_details)

[2022-11-15T18:28:32.676130+1000][CRITICAL][logger]][47576] You can't create a new User using this email!


failing to create user You can't create a new User using this email!
Ignoring: user with email:rey@skywalker.net already exists


Let's log in as the `Data Scientist` and get a different `domain` handle which we will name `skywalker`.

In [41]:
# ⚔️ Runnable Code
skywalker = sy.login(port=8081, email="rey@skywalker.net", password="jakku")

Connecting to localhost... done! 	 Logging into test_domain... done!


In [42]:
# ⚔️ Runnable Code
age_dataset_prt = skywalker.datasets[-1]
age_dataset_prt

Dataset: ages_dataset
Description: Ages of a group of people



In [43]:
# ⚔️ Runnable Code
phi_ptr = age_dataset_prt["ages_tensor"]
phi_ptr

array([ 64, 120,  31,  75,  54,  76,  60,  78,  38,  96])

 (The data printed above is synthetic - it's an imitation of the real data.)

Remember, we specified the `bounds` above so the synthetic dataset matches those bounds and gives us a bit of a feeling for what kind of data there might be.

### Private Mean

Let's find out what the average age in this dataset is by using the `mean` op.

In [44]:
# ⚔️ Runnable Code
mean_ptr = phi_ptr.mean()
mean_ptr

array([14.73111187])

 (The data printed above is synthetic - it's an imitation of the real data.)

Remember this is not the real value, but we can see that our shape has gone down to a single scalar value as expected.

We could continue to run additional operations but at some point we want our result.

### Logs

Before we publish, let's load up the logs for the `worker` so that we can see what happens when when we do.

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🍏 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
docker logs -f test_domain-celeryworker-1
</blockquote>

### Publish

Let's get the result of this computation by calling `publish` on the pointer with a `sigma`.
Remember the return value is a pointer to the result which if successful will be available for download.

In [51]:
# ⚔️ Runnable Code
result_ptr = mean_ptr.publish(sigma=1.5) # 1.5 is the default

Since `publish` can take some time we will have to wait before we can call `get`.

In [52]:
# ⚔️ Runnable Code
result_ptr.block_with_timeout(60)
remote_mean = result_ptr.get(delete_obj=False)
remote_mean

27.435222903256502

Interesting, lets compare that to our original values above.

### Compare

In [53]:
# ⚔️ Runnable Code
local_mean = ages.mean()
print("Real Mean:      =", local_mean)
print("Published Mean: =", remote_mean)
print("Difference:     =", round(abs(local_mean - remote_mean) / local_mean * 100, 2), "%")

Real Mean:      = 30.4
Published Mean: = 27.435222903256502
Difference:     = 9.75 %


### Epsilon Spend

That's pretty cool. So, what did that cost us?

In [67]:
# ⚔️ Runnable Code
current_budget = skywalker.privacy_budget
print("Starting Budget:", starting_budget)
print("Current Budget: ", current_budget)
print("Cost:           ", starting_budget - current_budget)

Starting Budget: 999999
Current Budget:  999690.035704
Cost:            308.9642959999619


**Note** if you recreate new pointers and publish them they `can` count as new epsilon spend so your cost below will change the more you keep running the cells.

### Under the hood

Okay so what just happened? That was all a little bit 🧙🏽‍♂️ magic but to understand something properly its important to demystify any magic.

We saw earlier how operations on `PhiTensors` can continue indefinitely until they meet another `PhiTensor` with different `Data Subjects`. At that point we get a `GammaTensor`. The GammaTensor data structure tracks the history of operations in a `graph` allowing us to attribute who's data was used at each step to produce the final output.

### Ingredients

As mentioned earlier to `publish` we need the following ingredients, the `GammaTensor`, a `DataSubjectLedger` for the `Data Scientist`, two `Callable` functions which will let us `get` and `set` the users `budget`, the desired `sigma` of noise, and two more variables, `is_linear` and `private`. 

- `is_linear` allows us to take some shortcuts if we know we are not doing `Deep Learning` type operations which often have `nonlinearities`
- `private` allows us to control if we calculate the epsilon on the real data or just the bounds

```python
# packages/syft/src/syft/core/adp/vectorize_publish.py

def publish(
    tensor: GammaTensor,
    ledger: DataSubjectLedger,
    get_budget_for_user: Callable,
    deduct_epsilon_for_user: Callable,
    sigma: float,
    is_linear: bool = True,
    private: bool = True,
) -> np.ndarray:
```

The first thing we do is calculate the `epsilon` for the current `graph` as is and see if the user has enough to `publish`.

### RDP Parameters

The next thing we need to do is collect all the parameters which go into the calculation of `RDP` (Renyi Differential Privacy) so that we can call `calculate_bounds_for_mechanism`.

<blockquote><strong>📕 Paper</strong><br /><a href="https://arxiv.org/abs/1702.07476">Renyi Differential Privacy</a></blockquote>

Given the `child` data, the `min_vals` (or `lower_bounds`), `max_vals` (or `upper_bounds`) and the sigma we can compute a worst case `l2 norm`.

```python
@jax.jit
def calculate_bounds_for_mechanism(
    value_array: jnp.ndarray,
    min_val_array: jnp.ndarray,
    max_val_array: jnp.ndarray,
    sigma: float,
) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.array, jnp.array]:
    ones_like = jnp.ones_like(value_array)
    one_dim = jnp.reshape(ones_like, -1)

    worst_case_l2_norm = (
        jnp.sqrt(jnp.sum(jnp.square(max_val_array - min_val_array))) * one_dim
    )

    l2_norm = jnp.sqrt(jnp.sum(jnp.square(value_array))) * one_dim
    return l2_norm, worst_case_l2_norm, one_dim * sigma, one_dim
```

The L2 norm calculates the distance of the vector coordinate from the origin of the vector space. As such, it is also known as the Euclidean norm as it is calculated as the Euclidean distance from the origin.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://machinelearningmastery.com/vector-norms-machine-learning">L2 norm</a></blockquote>

So we are looking at these bounds and thinking, what is the distance between each bound as a Tensor in the shape of the data.

In [55]:
# ⚔️ Runnable Code
from syft.core.adp.vectorized_publish import calculate_bounds_for_mechanism
l2_norms, l2_norm_bounds, sigmas, coeffs = calculate_bounds_for_mechanism(
    value_array=ages.mean(),
    min_val_array=0,
    max_val_array=122,
    sigma=1.5,
)
l2_norms, l2_norm_bounds, sigmas, coeffs

(DeviceArray([30.4], dtype=float64),
 DeviceArray([122.], dtype=float64),
 DeviceArray([1.5], dtype=float64),
 DeviceArray([1.], dtype=float64))

That seems to stack up, since we have a `scalar` value and the bounds of our `ages` were 0 and 122.
The distance is 122 because the `sqrt` of the `sum` of `square` is still `122`.

Of course this code can handle large `n-dimensional` Tensors in much more complex scenarios.

### Calculating Epsilon Fast

Calculating `epsilon`, the thing we care about the most can be expressed as the following:

```python
epsilon = alpha * l2_norm(q(X_i)) ^ 2 / (2 * sigma ^2)
```

However, calculating this for every single data point becomes quite prohibitively slow.

One approximation we can do is by removing `alpha` and re-arranging like so:

```python
rdp_constant = l2_norm(q(X_i)) ^ 2 / (2 * sigma ^2)
```

This means that for some `constant` we call `RDP Constant` there is some `alpha` which will provide the `epsilon`.

```python
epsilon = alpha * rdp_constant
```

Since we need to calculate these epsilon values for large `tensors` it is desireable if we can take some shortcuts. If we could somehow precalculate these `RDP Constants` for certain `alphas` we could quickly look up `epsilon` rather than calculate it each time.

### RDP Constant

First lets figure out our `RDP Constant` from our `RDPParams`.

We need to assemble the outputs above into an object called `RDPParams` which just helps us keep what we need together as one unit.

In [56]:
# ⚔️ Runnable Code
from syft.core.adp.data_subject_ledger import RDPParams
rdp_params = RDPParams(
    sigmas=sigmas,
    l2_norms=l2_norms,
    l2_norm_bounds=l2_norm_bounds,
    Ls=coeffs.copy(), # is_linear == True so we can cheat here
    coeffs=coeffs
)
rdp_params

RDPParams:
 sigmas:[1.5]
 l2_norms:[30.4]
 l2_norm_bounds:[122.]
 Ls:[1.]
 coeffs:[1.]

```python
# packages/syft/src/core/adp/data_subject_ledger.py

@partial(jax.jit, static_argnums=1)
def compute_rdp_constant(rdp_params, private: bool):
    squared_Ls = rdp_params.Ls**2
    squared_sigma = rdp_params.sigmas**2

    if private:
        # this is calculated on the private true values
        squared_l2 = rdp_params.l2_norms**2
    else:
        # bounds is computed on the metadata
        squared_l2 = rdp_params.l2_norm_bounds**2
    result = (squared_Ls * squared_l2 / (2 * squared_sigma)) * rdp_params.coeffs
    return result
```

Okay, so we have our RDP Params ready to pass into `compute_rdp_constant`.

In this case for the scalar data above we have:  
```
    ((1^2) * (30.4^2) / (2 * 1.5) * 1) ==  
    (924.16 / 4.5)                     == 205.36888889  
```

In [57]:
# ⚔️ Runnable Code
from syft.core.adp.vectorized_publish import compute_rdp_constant
rdp_constants = compute_rdp_constant(rdp_params, private=True)
rdp_constants

DeviceArray([205.36888889], dtype=float64)

### Calculate Epsilon

Okay, we are almost there. Now that we know the `constant` term, assuming a fixed `delta` of `1e-6`, we can optimize the remaining `alpha` term using `scipy` and the `Brent` algorithm.

In [58]:
# ⚔️ Runnable Code
rdp_constant = 205.36888889

In [59]:
# ⚔️ Runnable Code
from scipy.optimize import minimize_scalar

DELTA = 1e-6
log_delta = np.log(DELTA)

def partial_alpha(constant: int):
    return lambda alpha: alpha * constant

def search_alpha(partial):
    def fun(alpha: float) -> float:  # the input is the RDP's \alpha
        if alpha <= 1:
            return np.inf
        else:
            alpha_minus_1 = alpha - 1
            return np.maximum(
                partial(alpha)
                + np.log(alpha_minus_1 / alpha)
                - (log_delta + np.log(alpha)) / alpha_minus_1,
                0,
            )
    return fun


def find_alpha_and_eps(
    constant: int = 3
):
    search_func = search_alpha(partial_alpha(constant))
    results = minimize_scalar(
        search_func, method="Brent", bracket=(1, 2)
    )

    return results.x, results.fun

First we define `alpha` in terms of the input `constant`. Then we define a search function for `alpha`.

Finally, we pass these functions to `minimize_scalar` using the `bracket=(1, 2)` and `bounds` from `1` to `infinity`.

In [61]:
# ⚔️ Runnable Code
alpha, eps = find_alpha_and_eps(
    constant=rdp_constant
)
print(f"The solution for RDP Constant={rdp_constant} is alpha={alpha}, epsilon={eps}")

The solution for RDP Constant=205.36888889 is alpha=1.2572102721433043, epsilon=309.4281090883548


There we have it, our worst case `epsilon` spend to publish the `mean` of our `10` ages with `sigma=1.5` of noise added given our bounds are `0` and `122` is `309` epsilon.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://en.wikipedia.org/wiki/Brent%27s_method">Brent's Method</a></blockquote>

### RDP Cache

You may have noticed we `timed` the above calculation. It's pretty fast but its just a single value and we need to do this potentially millions of times over and over, while also filtering multiple times, until we can find a safe subset of `Data Subject`'s data to include in the final result.

Since for a given `RDP Constant` term there is an optimal `alpha` which provides the `epsilon`, it's a perfect candidate for a `cache`.

Rather than do that Brent optimization above on every single value, every time, we could just lookup an approximate that we previously calculated. So that's what we do.

Let's see how it works.

In [62]:
# ⚔️ Runnable Code
from syft.core.adp.data_subject_ledger import load_cache
cache = load_cache("constant2epsilon_1200k.npy")

This `cache` holds a 1.2 million precalculated `epsilon` values for a range of `RDP Constant` terms.

The only catch is, they are not linearly spaced. To provide a better use of the `cache` for different calculation types we split the cache into two parts. Constant values `<= 50` and values `> 50`.

Don't worry, we have a convenience function which does the re-mapping for us.

In [63]:
# ⚔️ Runnable Code
from syft.core.adp.data_subject_ledger import convert_constants_to_indices
cache_indexes = convert_constants_to_indices(rdp_constants)
cache_indexes

DeviceArray([500154], dtype=int64)

This tells us that if we want to get an approximate `epsilon` from the `cache` we can use index `500154`.

In [65]:
# ⚔️ Runnable Code
cached_epsilon = cache[cache_indexes[0]]
cached_epsilon

308.9642958587137

Look at that!

In [68]:
# ⚔️ Runnable Code
print("Calculated Epsilon: ", eps)
print("Cached Epsilon:     ", cached_epsilon)
print("Difference:         ", round(abs(eps - cached_epsilon) * 100 / eps, 2), "%")
print("Previous Spend:     ", starting_budget - current_budget)

Calculated Epsilon:  309.4281090883548
Cached Epsilon:      308.9642958587137
Difference:          0.15 %
Previous Spend:      308.9642959999619


If we look back at our `spend` we should see that it matches up.

### Differential Privacy Ledger

We mentioned earlier that a unique aspect of `syft`'s DP system, is that it keeps track of the cumulative $\epsilon$ spend, for every `Data Subject`.

This is referred to as `Individual Privacy Accounting`.

To achieve this we need to keep track of the noise added by which `operations` in what `order` on what `data` and by which `Data Scientist`, for the life time of the system not just the Data Scientists session.

<blockquote><strong>📕 Paper</strong><br /><a href="https://arxiv.org/abs/2008.11193">Individual Privacy Accounting via a Renyi Filter</a></blockquote>

The way the ledger works is every time we call `publish` we figure out what the `max` $\epsilon$ is for any data point which any given `Data Subject` is contributing to the calculation of and we `accumulate` that on the ledger so that the value continually increases over time.

As each `Data Scientist` has their own ledger the `Data Subject` `risk` can be calculated on a per `Data Scientist` or per `Domain` level, and eventually risk could be calculated across an entire `Network` of `syft` `Domains`.

### Data Subject Safe Guards

We are building a tool to provide interpertable information about the probabilistic `risk` that `Data Subjects` face to `Data Owners`. This will allow them to set `Safe Guards` such as per `Data Subject` and per `dataset` limits.

Once these safe guards are in place, during `publish` the `DP Ledger` will query the `Safe Guard` system and determine if the current `operation` would exceed the desired `risk` limits for any given `Data Subjects` in the derived data to be published, and if so; `filter` those containing overbudget data points; in the same way we currently prevent `Data Scientists` from going over their budget with a `query`.

<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRu4jAABXRUJQVlA4WAoAAAA0AAAAYwAAYwAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBIywQAAAGgBUm2advqqYNn27Zt27Z5bevZtm3btm3b
PthrYvTH0d5rzRvvNyImACVU1gJQMy6y9d6HHNWvf//+gwYP6nf4/hvNOwMAWKNQ
fW0UMNvGI65785f2wnvvQ0gioej484dXLj1qzekAGF0t7QA07Xbzu/+yq4gIKd2y
299eu3KPaQFlVWVME9C64vEfRFK8j0n6mGIoPMn41QkrWMDpSmgHzHPoM0ImH4V1
lhRDIuPt200LmPIpDSx+4kdkComNTiGRzw+dCcqUzALznv4zWQRhGcUH8r1DHJwq
UxNaJ/5J+sjSSgjkEytDmdIYjXWeJaOw3DHy3+GAK4kDRneyEJbfkw8sBqPL4LDA
U2TBSkbPP/aE0Q1TGht9zSKxqp5hApRukNY4omDBCqfEy6BNQ7TCCWRgpaXGK2FM
AzSaL2VMrLh08jpoUzcFdSW9sPoFr4DW9dI4jQWzWPAWaFOfJuySkuSBnbwURtfD
YvGfGJhJqfE4mDoouJdZMJsSuB9c3yxOZ40ZTfx7Jdi+OOxEYVY9X3TQvTNY+hfG
vLDg8bC9UkrdzIKZlcR1YHtjsLVEyQ09H7G6NwqPMDC/kUfA9mSwXkzM0sczQPfg
cAdjjhg5HK47iw3bKJn6fCGYLsrhChbMc8FRsF0MVvmbKVORr04PBUDjcnrmOnED
GEBjiXaRbAWe0cWiHwOznfhJs1LQeIIxX8K2dWAMVmljzgNPhXMYw5CzyJdbtMNd
eUv8bQ1glrcZcyaeBwCr/cqUMxY8GdiPQbLmeZvFFHpmPfKjeXFt7kSK7XFH7pg4
SL3CmL0p6qupwKnmF6bsnen+oWTvLNc2Vej4n3C2a5sKnGn+ZMre6ep7xuydpN7P
nSROwL0MeWNiP9xEnzdJ3B1n5S7xp6UwMHeBz8+I9f8RyZrn1cBcHzBmreAUAI8w
5EwC9wFwYt4Sf18TwKY15jzwpWYNuA+ZsjYZDgaTGPIl/GdFGBisxCTZinwFGlCY
4S3GbAUOggVgMZE+V4k/zgMFwGDZn5gy5XkiDAAoh2tY5CmljjVhu8Bio3ZKljyv
hEP3Flcy5kj423KwPWgs45NkKPB6NKFnjUvo8yPy77LQvVrgG4bs1DgFBr1tQr/8
eH40K3SvlMHd9HkR4W5w6L3B4v8yZcXzGljVB1gcypCTyN+Wh0afLc5mZz5EeBAM
+m7Q8gI7cyGRxyul6gCL2d9mZx6kxpOgFOpqMec79FkoeBmURp0dZnyMRfXE80ql
NOruoJ9jrWqpxsuhNRrYhNlepZdKxcRLlTJoqMUMdzGmCnnWJgMaDbbA2aSvSvL8
ewdohYYb4JD/GGIlCvK5FeAUSqg1lrub9OVLBTsmKliU1AKD/2CKUqoUyCdWgzYo
rTFY7PyCyUtpoie/OrIFTQolVg5Y+6HA4FMpYkH+cOIcUA4lNxrY6bFEhigNSiGS
Pxy3EGAUym8s3Kan/UwyhZikLpJ8IMkXhy4GWI1qOg3MN+rlPwPJ4L33MYmQIhKD
994nkv99cukW0wNOo7LKGACL7nXDOz+z+xhTSjGx29rnDwxarRWANai2NgrALKvu
M/qsq+98+3d2G757/qYrThqw5cIAYIxCBrU16Na0zLnsqmuutfpKi87YYtDVOqNQ
QgBWUDggCA8AABA3AJ0BKmQAZAA+MRKHQqIhDBXrPhABglsAMURaFreJH2b8hPZP
qn9Q/AX5N/C/vz6A8p/xn9R/0392/Iv3tf4r2SfnP/ge4B+n/++/wvWN/a31AfrF
+yvu1/2T9ufcT+vv6yfAB/NP63/7vW89gj9wvYD/oP+K9Ln9tvgy/az9rfgV/Yf/
8ewB6AHCl/1Xtc/zPhP5L/Uftj67OQvrozUfgf9h/XuK34/6gXsnemQAfW/vlNV/
wF7AH5g+VL4SXmPsCfzb+9eiL/0eZ76o9g39a/+p66Xst/bj2KP1pU9b/HlT29UP
gOk1ocOhd7KIwhXsujqDwBQJVHh+WGmqNYv0ssvAOxFX7j6FQEG12XhHa2Y1DcUc
Ld1YCwWb64/7M1V2xIoEDnZSBBYnHb35VveydM/1bJpXyfYPngm3C8ux7bo5NkAU
RNzwHuGJY4mOq8wXB8ycpUe3ajsDMUMiXoIJwT9fxh0KdpHtGDqnUajPBddA/Jcq
W+SRxl67DGnbY8pucInlJhbjL/4e+rQyxqYukbtsOwecsmS88YjgJs9aQ3ddL5C7
0/Nd1CV1UDf07f/D5D0+2Xubgcnl77D/ig9AAP7/1feo9QUPI/kGnH426lUOcnBc
KFvgrKtbvfbSdmGYern/Sw8M/+AnpouCyB+fElMRnV0jdJzZ/XWVwC8+m2I2ncOM
ogp3NQvqk0bjdJU+Ne3dbAv8L3Uc23S5/ExF9VnHEK2a1yENQUqnX4N+6SOzD//R
rFCQjcOMGBqcnuL5tXvwow095jgHiBCdzkNLoLo8S8z6CgHeuBibiLGOWe6YAXQ9
DChogooOeLykDQLDlfj7VCsy2AluPd1D90xPKLYQzdirZCRfCjtLu5jaeQ0v26Vr
n7NMzZ+ZK9t2YvdCRbxkTlJfhKt1ImZjn6VdMG40siS9W/HbqU3x68wvIPAp4/xe
fX/Obejtp/x3AwFKxCep5BY8YIj4r2bYVoEXmhCOOCzSOz5HZnDi9V9R/LmKcL4P
Z0Q9xjwyQOzPbVbMRt6JPLWkSfOyQmnSCN2EO3xmzkqgr/QqIGv2AJErApMZFTOr
VRmoYzu7aYe3TYZLCXs97iNMf5uRfhaR/x3mBGBq0GX4HLBb5vG8usx/oy+2Vw+K
29/o71d8686Kc2c6w/oAkz6v31IBwJTr/xa5nu1bwADKKLSgvqCMFumhn5UtYJwN
Vb50C823r/e6vDBIBmMP+3NrgM8399iBczXadCjwtP0RqRxF+npIqhZ56tsQ0Gms
cBxWqDHWr2QkewSSbNy2nqWGXlSSR5gGzI997k8SW4tFwaHuxaNkgbmGxv2A1Wqf
2XeqVYkaHcxNIsewC7JJSx+mFhAi+4Y3sMg2j6iCXzMcbtL7iTPyEvDAI5L/td7X
06o5FKk+v4TUgZ1jmVR62NZQZu87okmHXiUblPMqlX+8UXIoGcLG28rrYmWLVlf5
4PGWqN37lwapi9wz7+F/haCbqVUKLXBvJ88ukSMUkKal9zvT/zP7aM7jLp7RMWnO
DXu3JgARDKErj8cMXo3hjCzlPyvSTOV9BwUkpWcTulMoADESo4d+KDPlVAed9Bg2
rAjmTsHWNKJK6FFYfDauJT1RWOoVDZnl60/JlswY0kx/DvE/fQOcC4xz/oglm/bf
V09iCYbASWOZM05eYnoRRXgrZxysDFHrA4qvTUHgL/w2lHLHdd3tF7lvVIP7c8Hd
Mhoe6yB8fktF+o6iRuctogeB4Wk6WAlzgHavfCbQZXlMU5+xuLnrbHBedNyGa3eR
2LARv8PQ093GK3qGfyadVpnbwMxzg9MSw5YYkePnnRaZ4Q9sWInFXzZKndFkItc3
daJt/Hd21Bq6x07kEpk1DkVJ8ShasNDERuqiaT/R27Hp7r/w11I2x2UYv2VdJ9B4
B165OqvMYtFhiM/Ge8FFD0QYHmOmUe6accn3xxsmza03FlQQl2/GB+DL+9yuPc4z
fjydZM6s0FT/Ydh1YXbB8CIGKCfjCHOIR8fEz7AuNGmKE9hcPrYZUKGKH2QuyzxO
0Dr2AWgdCTxDakmPSRhzGS4/AUDjRhSPUJhw8v5RRTLhxndqmRi7Bs1hCEi3VX/4
qhhrjK2BCW7KS60BBh2XRoEMfnBgEIg0D60qjwGIbq5qk0OoqwrACp4WN9ToTjmT
gBNLTOlydkoxVp7uad1rPymchKukOhyJB3I+pwMKhROsAJdHdFkgziVWR5psBWGR
TuJ68ySGCcDp5wYKhz55aaVO/Vx3lkXxTXMpvCqGd8hZlQ+vzJNhTPhi1ur8b+DQ
AKSvK6zyqMuK+p3/xfngEgKkfl0bh2S1aZeTppZbAMMLcBK/0W692Dc2tYFOGGs5
lIDMx1N08E557m85KYSqvCt9hmnGZQD/oS8idkwdO1VXgv53nP5PLiQ/JXXfPrVt
ytwbRDVEeGTesyU+Dfgsgl39jtsWuWyBaGuzPi4QLo6qMJyPivL2/dxqGxpzY0Ou
rFTdVk5MrsuuTkoKPZVlDfGGsbgFL4EmoDHHMnVF/KEB346+Hg+2IIPYxuq/cjH4
Evv+OyKj127Hgw99Xq6wHtzBsRs+DvvQ23gq4n5l+dFnssDwfD2hZkc7VvJxcNew
YL0TAfNgX8EfCnjdUg4LQM09RdNh/iWwogGcQHzgT6tJbIdWNEJL7x0uP57PFwYJ
b1gtxXV6kduTuNfcjCRQtuX5nvYBdVdMR6sL0h9nUX/twsaWic3/mNE+vvLNSsJ4
Up4H/SIqNM37xxphxzFyN4wW8/aqyF9Z8rsdQVMLmFryrwcJCcEi1Ub+X6oX/zQo
YHQlNj5WMa55Ewl74oGn3IV1YAuOf1uE0b5ccxADNNrMVd6MQhozMhmqL0KOOD99
6ngdyhKuMvv8TlTc2cZ3S8vjGaa1haI+hrRye/7b2pgFMpE5C86FDLo/Wstrz0pd
+B/X+Vf3DnbJ2kIRHSHRXsJW5DAaVhTLSG7dParbM/aC34xSIdHfv06Oc+z9n25d
Q7rOgo6XRtx7i2/u+qev6b1GACUFKEfTNvg4d+VSmO3EXBP7pWoKSTkSyYjojGOI
N02VP/QW89M/lMnxH3yDiQb0oMnZyauWmDURCpqTH/J+RPADITYRNPy9tPFuShTS
uRY9VWnHisoa/GTpUkEtTCPWDT+gIYCL6ZunSK7/nQDsDbEogIsMfLRgcfPzw7sF
IlF7NU/o3nVSeDNNC3LoJj57q44rP0Buf4XTZmgNDSUq9GzSIywuU8ANSCzws2Oq
cSQMBJqTfaMyqszJ1UpZrwW6KS2w0dTo/ZIslD9MJ76fYctwUy5e3aExmgtwOF7b
EHMqvInwDH68cwfzRp8kdj+kTjPLQ9XvzxTf9gypE7zAuczgOEwnuQMd1Wkahm7s
2B3b9jDT0c49CqMrR9c9CIo8j4jzO1FdrFo2vLMBBg3++SFAFNWq1u1zaUOrIAwJ
+5hXeGhcvGV+CDrPbH5Qu6DquPxg6M3r+BidXkIGqwEvD6y36T/WWXtELSGc8EFX
zx8H5Ec+2PcwwPAocjpP02D2KRM8OAAggO+8L3gm4D9WzBoX7wPJ6bAhKuslybaq
PfSCyoEkz4SakURUVh0j01qEJXDPD0lCmeFp0nBWYemxhN83+2ecKiAIYPeXObPb
bOXpVpNH9GnCKMpgBpTMMx1oPCugeGpX8LHau6GkdXLzzHDSd/ik7rJl+LtMye3J
YcdtDZwqDMkQaqdQ9zALPauJxQIkmzxF/lM/rEf/P11Wt8VxkLlN54VQl/iHKOVQ
XL6F48bocjl6MK2YYy8H/6EY8DFz2MU5I/VRz7OgbdLaiLATXe9+bVD9H5RaQ+MX
mzNvrucPcuyK8/CFYU8EomtNTNGZ8fMYgk1yIqq6glClIZisc0WE9Zen4/WVv8Kv
OivBqdbFbIhrR048uuQ1SmUikuBa7+AQy9ipc5JxA0vhqp87zDQSmou9lJWkPec9
RF0SyaJxS8A3CPH2THfAoiZqRzc+rCXVQI1iDyJpmX5U4TqqEv4PLWWwcyqGy+iK
NmSR96vqGLY2uxwpLMDJUY0poSph9yiSm3KefyEi9MrPTE00nC2QOR6CqV3+YYhc
Y9X6LX1hCS7ih/vkfADzwAu6a+UdLg/W4AK314nj/hvv0SXD6FXGgydMm//1Tn49
J+Q3xuAxtD9gv9Qbg3Jg5BSV8XN+39brnUnIXF+3aNCVZUnehA2t6SR1k1RnwUVd
UKl2wsW9Dev+xeB/hFy8OvAX6JS8V/MtKtm1wkI2waF0q6zqsw2LMNLOtVVNwWMV
gJToWSZzsWyPgI9jGhJyAyLkV8dAjMIEP+TXNSju4Q3z10WN8oAxrvEbmYZgQzxs
xmAY0pZ8plzxLlV2Wgvx9l5KkR7gTE4jp9b3/7gr1/rfVvPma+rQTDs72lvk9VD2
g3hTyzF0j6ec/0/elex1vYdm6nouY4MDGemBO2fllXfCexrRKbL7/aOkNY81qCs3
XXe2vGJ/kq51rSo7HzYwt1IoMwam2qeRAMrCufL056N4HU1UKU63mt36vKfyz2Ka
2LhcfnLvey6qSGj6MtjIswk3en26VEnrbFo5TsDw1IwPTFUSPD6bvyD93m5NBopv
Ptxirp8n7HsQO4jySfzYYFLR3h6xykmyHWQTlWvnelv8k6RrdhaspBYhIXnzc94Q
aL2vQYhVFYR52Mss1Ao5LAbbIu/ftmAgUaja3V7fMPn9e3N/lKpKeXr6JF5jCJa5
wvqTgsKkFJ9iNrd3GFga8oWPGJWvW6Ezm/XjFB8gKIJ6+l7RsQ46C325R78uRa6w
SEPn3qj0a+McQ2D4hnUOEMzARLVggy4bPokkeSRSVIx+EcZDOYsplWuhmb9z9bc6
qPO/Q73EFasUJE8/+LfJHVff3r/+NFFQ5AAGn/QAxD2Vqc99t/u7ZD9TY3q9qZvH
educuKcody3wEv4ODAWiyqYnIycdbE5/D8fGLHCEVmSMgaTCBCH2VRXYgwG/GjqT
8kyZDcbXbezrhnlpKvhyz/7ykR6ICetW7bAnZNYFBIj0SABOjLsgIHemeyscxKLZ
SKVB8O0abKz5h8ABj3B8VSCcEnHtKbZVVKACIV4TzeY9t85rRsH1ZXodQn2I/dIq
SBWO6vOnxsn/4ddeXygAAAAAWE1QIJwDAAA8eDp4bXBtZXRhIHhtbG5zOng9ImFk
b2JlOm5zOm1ldGEvIiB4OnhtcHRrPSJYTVAgQ29yZSA2LjAuMCI+CiAgIDxyZGY6
UkRGIHhtbG5zOnJkZj0iaHR0cDovL3d3dy53My5vcmcvMTk5OS8wMi8yMi1yZGYt
c3ludGF4LW5zIyI+CiAgICAgIDxyZGY6RGVzY3JpcHRpb24gcmRmOmFib3V0PSIi
CiAgICAgICAgICAgIHhtbG5zOnRpZmY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vdGlm
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczpleGlmPSJodHRwOi8vbnMuYWRvYmUu
Y29tL2V4aWYvMS4wLyIKICAgICAgICAgICAgeG1sbnM6eG1wPSJodHRwOi8vbnMu
YWRvYmUuY29tL3hhcC8xLjAvIj4KICAgICAgICAgPHRpZmY6WVJlc29sdXRpb24+
NzIwMDAwLzEwMDAwPC90aWZmOllSZXNvbHV0aW9uPgogICAgICAgICA8dGlmZjpS
ZXNvbHV0aW9uVW5pdD4yPC90aWZmOlJlc29sdXRpb25Vbml0PgogICAgICAgICA8
dGlmZjpYUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WFJlc29sdXRpb24+
CiAgICAgICAgIDx0aWZmOk9yaWVudGF0aW9uPjE8L3RpZmY6T3JpZW50YXRpb24+
CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4ZWxY
RGltZW5zaW9uPgogICAgICAgICA8ZXhpZjpQaXhlbFlEaW1lbnNpb24+MTAwPC9l
eGlmOlBpeGVsWURpbWVuc2lvbj4KICAgICAgICAgPHhtcDpDcmVhdG9yVG9vbD5Q
aXhlbG1hdG9yIFBybyAyLjQuNzwveG1wOkNyZWF0b3JUb29sPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yOVQxMzo0OTo1NysxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6TWV0YWRhdGFEYXRlPjIwMjItMDktMjlU
MTQ6MDE6MzcrMTA6MDA8L3htcDpNZXRhZGF0YURhdGU+CiAgICAgIDwvcmRmOkRl
c2NyaXB0aW9uPgogICA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgo=
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">🛕 Trial of the Spirit</h3><p>Complete this task on your own computer to progress to the next session.</p>
</td>
</tr>
</table>

To internalize how `Differential Privacy` works and how it is implemented in `syft` one of our Mentors has put a branch <a href="https://github.com/OpenMined/pysyft/tree/padawan_trial_dp">padawan_trial_dp</a> up on the PySyft repo.

You need to do the following:

- checkout this feature branch
- create a new branch named something like: `yourname/trial_dp`
- implement the missing `__mul__` op so that it works on `PhiTensor` and `GammaTensor`
- remember copying the answer from `dev` just means you don't learn so try to do this by hand
- make sure the linting checks and tests pass
- to run these tests run `tox -e padawan.trials`
- push to your own remote
- open a PR (The diff might be significant as the functional implementation has been changed in the latest dev branch)
- Report to your Mentor

## Brain Food

In [69]:
tweet_embed = """<blockquote class="twitter-tweet"><p lang="en" dir="ltr">One of the goals of Privacy Preserving Machine Learning is that &quot;All your data are belong to you&quot; <a href="https://twitter.com/openminedorg?ref_src=twsrc%5Etfw">@openminedorg</a> <a href="https://twitter.com/hashtag/ppml?src=hash&amp;ref_src=twsrc%5Etfw">#ppml</a> <a href="https://twitter.com/hashtag/AI?src=hash&amp;ref_src=twsrc%5Etfw">#AI</a> <a href="https://twitter.com/hashtag/MachineLearning?src=hash&amp;ref_src=twsrc%5Etfw">#MachineLearning</a> <a href="https://twitter.com/hashtag/syft?src=hash&amp;ref_src=twsrc%5Etfw">#syft</a> <a href="https://twitter.com/hashtag/opacus?src=hash&amp;ref_src=twsrc%5Etfw">#opacus</a> <a href="https://twitter.com/hashtag/PyTorch?src=hash&amp;ref_src=twsrc%5Etfw">#PyTorch</a> <a href="https://t.co/ndDuyH2Vd2">pic.twitter.com/ndDuyH2Vd2</a></p>&mdash; Madhava Jay (@madhavajay) <a href="https://twitter.com/madhavajay/status/1333963102694510593?ref_src=twsrc%5Etfw">December 2, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>"""; from IPython.display import HTML; HTML(tweet_embed)

## 📚 Homework

To maximise your understanding of the material, we recommend you review this notebook on your own time.

<blockquote><strong>🛕 Trial of the Spirit</strong><br />Complete the Trial of the Spirit</blockquote>

<blockquote><strong>🎥 Video</strong><br /><a href="https://youtu.be/4zrU54VIK6k">Privacy Preserving AI (Andrew Trask) | MIT Deep Learning Series</a></blockquote>

<blockquote><strong>🎥 Video</strong><br /><a href="https://youtu.be/lg-VhHlztqo">The Definition of Differential Privacy - Cynthia Dwork</a></blockquote>

<blockquote><strong>🎥 Video</strong><br /><a href="https://youtu.be/MWPwofiQMdE">OM PriCon 2020 Tutorial: Differentially Private Model Training with Opacus - Davide Testuggine</a></blockquote>

<blockquote><strong>📕 Paper</strong><br /><a href="https://arxiv.org/abs/1702.07476">Renyi Differential Privacy</a></blockquote>

<blockquote><strong>📕 Paper</strong><br /><a href="https://arxiv.org/abs/2008.11193">Individual Privacy Accounting via a Renyi Filter</a></blockquote>

<blockquote><strong>📕 Paper</strong><br /><a href="https://arxiv.org/abs/2201.05964">Visualizing Privacy-Utility Trade-Offs in Differentially Private Data Releases</a></blockquote>

<blockquote><strong>📚 Learn</strong><br />If any of the discussed technologies are unfamiliar to you, it's worth reading the attached links or looking for tutorials or videos that explain how they work in more depth.</blockquote>

<blockquote><strong>💻 Code</strong><br />If you found any issues during setup, discuss them with your Mentor and if applicable try to fix the issue or text in the notebook and submit it as a PR.</blockquote>

## BONUS

## Mnemonic

You'll never forget how DP works if you just remember this simple `mnemonic`. 

Results protected with `Differential Privacy` should be indistinguishable:   
`With or Without You`!

#### With or Without You

See the noise added in your eyes  
With `Differential Privacy` by your side  

Ill `async await` for you  
Not enough just to utilize `FATE`  
Extend `RDP cache`, she makes me wait  
And I perturbate without you  

With or without you  
With or without you  

Through `Apache Storm`, we reach the shore  
You gave a `Data Lake`, but I want more  
No more waiting for `Data Owner` review  
With or without you  
With or without you, ah, ah  
I need `Syft`  

With or without you  
And you give `epsilon` away  
And you give `epsilon` away  
And you give  
And you give  
And you give `epsilon` away  

In [70]:
youtube_link = "https://www.youtube.com/embed/ujNeHIo7oTE"; from IPython.display import IFrame; IFrame(src=youtube_link, width="560", height="315")

I'll show myself out... 🙃

<br />
<br />
<br />

<table width="100%"><tr><td align="left"><a href="05-hagrid.ipynb">Session 5</a></td><td><img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAASwAAAAKCAYAAAAO/2PqAAAAAXNSR0IArs4c6QAAAMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAABAAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABIAAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjMAAAAEkAQAAgAAABQAAACyoAEAAwAAAAEAAQAAoAIABAAAAAEAAAEsoAMABAAAAAEAAAAKAAAAADIwMjI6MDk6MTMgMTY6MzQ6MDgAEulHAwAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7FpVFh0WE1MOmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwPC9leGlmOlBpeGVsWURpbWVuc2lvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjMwMDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDx4bXA6TWV0YWRhdGFEYXRlPjIwMjItMDktMTNUMTY6NDc6MTErMTA6MDA8L3htcDpNZXRhZGF0YURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTEzVDE2OjM0OjA4KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpDcmVhdG9yVG9vbD5QaXhlbG1hdG9yIFBybyAyLjQuMzwveG1wOkNyZWF0b3JUb29sPgogICAgICA8L3JkZjpEZXNjcmlwdGlvbj4KICAgPC9yZGY6UkRGPgo8L3g6eG1wbWV0YT4K/EIXYQAABW9JREFUaAXtWl1MXEUUPjNz7+6FZRcJbA1VUqgFW7EikJQWbeoWW6OGxBQRiD7gz5Np4qN/L/hgYvBRY2J88sVGaKq1NYoP1TaaFJXSRlNbBAoVXZF2KbCw93fGM/eyZaV92gcLzUx29sy998y5Z7698+05Z5fAGmw9PT00kdhfx7k1m0gkJrIudr1zttYDr5loZDcIiFFGqbxGCMmqKKkQUAisQwSEEL7XgnM5ygjuDRKqfT+dPjr8XU+Pm12Slh2glLteziLt7e10ZmbmBhaIx+Oiv79f6nDsq5vUp7W1tcwwDDE0NOTlKqBNMj4+Tqenp7WpqSkHr/lOlJWVRdvb2p7WmdMykZw9OvjTr5eppr9gFOjdqVQ6vbu1+8vyva+MMU5amcbqQ3qIogRC5e0IShTBWnNvp8YKAYXAekIAt7PgyxsZhet6Hdz1oJw89Ut775Pvp/+Y+fir956wSF9fX8GhQ/07KCUlGLm4VLAcMrpObP7ShWAmmkpxzjcmWlpqKysrN+paaNO5s8P3mLYV454XxmiHVlVWwVImw8fGL3HwnRAyCqIcLTNGMkzXroY0fU4IThcWFjcbRriypLQERn8fA6br4vHH9pPy8nIwLRM+O/YNOJx4bMdLeImBplFAX/2oSnLxDay6nj4k5atCQCHwXwRwQ+PuBuQYsG0PpYDF+Qx4Lv/goxe3HNQ4J+13lMQ+xKjIkLGY53kQhGcCpQxe/LcViYYc14WG+gaIxzdAJBIBx7Gxy6ApaNvu2wbcE5DY24IGcP5yC4b+8ebsOenQwsI8mKYJTTt2QllZqc9BnscBXYLnn+uAaCwKpHQjzCBd2kh6LnbJqlLRt7Zyi6xZJRUCCoH1hIBPVAFdyOSJYeZkMIAI5oB3GgDFGrys//3JYe304OmHhs8MG2Ej7O9+P4NEZgk4QJKWHPkcEtSK8FBe/XpgABofrAMkO7iW+gdDONfXlfPHRylkTBsmJ//EOYETudjllpyy9aeAzFa0MPrC4ExATXW1m2h59EcMvJrjuQnsiqoaKQQUArcrAstEZnsEdtdEWrVNVZtOHSht6wiHQsUBech0S64+SLtycaBY45Y6sldU3A3ToycgHbIwdDPBtlwkGAKFBTqkFjWMhIqgpmafryttSOKTdrPFteBcYF2eSyaTmOpRjLDiMD8/78+7NjcH1dsbfkhZdA/F0Ioh7WpIv5KBNexSSpv4Uu0WI6CC3Fv8AdxGt8fkChx8oKR08W0+vQSzV2Zg5NIkaHOp1KfHjx//ORqNGq5LMAhzJKnclAPkeWxFjid2dXV1PlukF27VCyMsFjV9siI43QgxmE0LJDENsMi+krZlAc19svEujDKwXWfk5LcDvYWhSLJ+Z3PthQsXHzAtZ4tp83tJc/ceftUBnRHQsMuSmC9xgPUw9FURVhbaWynll5hqCoF8EcjmdLLOLR8lWSqSVOFiaclyKNgLURi7YhzL+ylrbGzU6+rqsGBulFbfJYq1UKgkHgtXlUQNOHLir99Mx5m+nExi1QmbLG/p/sh/o55HGWPRSCy2oaKiwrp4/vzJgYGB1IpGMHqk7eDW2PbOtyKxomcKigrAKAwDw6K73BwUySqI+IKUU+2X1ej9v8fyi0M1hUB+CPg/yl3PviRZYZEduweW6YC5aEFmcan3yBtNr+ZNWPk5ls8sQTp7zzwsKHlT1/VmwkgU2c5PD7N/bUDKysewmqMQUAisGQQkFeE+xpf8hdC1sSYOwnQdb8i1zXcPv9b0BcZeN0/91swaVjly4O1T5YwYNfg/rPvR8yZcQAzTVP9/WatU1aFCQCGwjhCQ0RQ2gUFIBkOtIRDsnOVlRj5/fddE7jL+BVnXK0JzkMN9AAAAAElFTkSuQmCC" /></td><td align="right"><a href="07-secure-multi-party-computation.ipynb">Session 7</a></td></tr></table>